In [1]:
import pandas as pd
import scipy
import pyreadstat
import pyshs
import matplotlib.pyplot as plt
import seaborn as sns
import prince
from statsmodels.stats.proportion import proportion_confint   

import scipy.stats as st
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics
import numpy as np
from scipy.cluster.hierarchy import dendrogram
from statsmodels.stats.contingency_tables import mcnemar

C:\Users\rana_\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
dict_data = pd.read_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\SLAVACO - Vague 3\SLAVACOv3_CAWI_COMPLET+POIDS.xlsx",
                    sheet_name=['codes', 'labels'])

In [4]:
codes_data = dict_data .get('codes')
labels_data = dict_data.get('labels')

In [5]:
labels_data 

,CaseId,STATUT,poids_init,poids_redresses_stand,INT01,Q01,Q01_R0,Q02,Q03,Q04,Q05_A,Q05_B,Q05_R0,Q06,Q07,Q08,Q09,Q10_A,Q10_B,Q10_C,Q10_D,Q10_E,Q11,Q11_1,Q12,Q13,Q14,Q15,Q16_A,Q16_B,Q16_C,Q16_D,Q16_E,Q16_F,Q17_A,Q17_B,Q17_C,Q17_D,Q17_E,Q17_F,Q17_G,Q17_H,Q18,Q19,Q20,Q21,Q22,Q23_A,Q23_B,Q23_C,Q24_A,Q24_B,Q24_C,Q24_D,Q24_E,Q24_F,Q24_G,Q24_H,Q24_I,Q25,Q26,Q27,Q28_A,Q28_B,Q28_C,Q29,Q44,Q30,Q31,Q32,Q33,Q34,Q29_1,Q29_2,Q29_2_PRE,Q29_3,Q35,Q36_A,Q36_B,Q36_C,Q36_D,Q36_E,Q36_F,Q36_G,Q36_H,Q36_I,Q36_J,Q36_K,Q36_L,Q37_A,Q37_B,Q37_C,Q37_D,Q37_E,Q37_F,Q37_G,Q37_H,Q37_I,Q37_J,Q37_K,Q37_L,Q38M1,Q38M2,Q38M3,Q39,Q40,Q41,Q42_A,Q42_B,Q43_A,Q43_B,Q43_C,Q43_D,Q43_E,Q43_F,Q43_G,Q43_H,Q43_I,Q43_J,COMMENTAIRES
0,10,Complet,25416.857072,1.015050,Oui,1946,65-74,Un homme,Provence-Alpes-Côte d&apos;Azur,de 2 000 à moins de 20 000 habitants (exemple ...,Vous êtes à la retraite ou en pré-retraite,Ingénieurs et cadres techniques d&apos;entreprise,Retraités,"Diplôme d’une grande école (ingénieur, commerc...",4000 euros et plus,2,Non,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,France,Oui,NaN,#NULL!,Vous connaissez un peu les essais cliniques,"Oui, plus ou moins",Tout à fait d’accord,Plutôt d’accord,Plutôt d’accord,Plutôt pas d’accord,Tout à fait d’accord,Plutôt pas d’accord,Plutôt,Plutôt,Tout à fait,Pas du tout,Plutôt,Pas du tout,Plutôt,Plutôt pas,Assez,A peu près autant de bien que de mal,"Oui, et vous pensez que la chloroquine et ses ...",Assez bon,"Oui, j’ai un problème de santé",Très facile,Facile,Facile,Tout à fait d’accord,"Ni d’accord, ni pas d’accord",Plutôt d’accord,Plutôt d’accord,Plutôt d’accord,Tout à fait d’accord,Plutôt d’accord,Plutôt d’accord,Plutôt d’accord,Vous connaissez seulement le terme “CBD”,Un peu,Jamais,Jamais,Tous les jours ou presque,Jamais,Vous êtes déjà vacciné (une dose ou plus),Plutôt pas favorable,Plutôt favorable,"Oui, tout à fait","Oui, plutôt","Oui, tout à fait","Oui, tout à fait",Mars 2021,Non,NaN,#NULL!,"Oui, mais vous l&apos;avez désinstallée",Difficile,Facile,Difficile,Très difficile,Difficile,Difficile,Difficile,Difficile,Facile,Difficile,Difficile,Difficile,Facile,Facile,Difficile,Facile,Difficile,Facile,Difficile,Difficile,Facile,Facile,Difficile,Difficile,Les échanges avec vos proches,La presse écrite imprimée,"Les réseaux sociaux (Facebook, Twitter, etc.)","Oui, beaucoup",Tous les jours ou presque,Vous votez à toutes les élections,Les Républicains,#NULL!,Plutôt pas confiance,Pas du tout confiance,Plutôt confiance,Plutôt pas confiance,Plutôt confiance,Plutôt confiance,Pas du tout confiance,Pas du tout confiance,Plutôt confiance,Plutôt confiance,NaN
1,1000,Complet,25416.857072,0.678083,Oui,2000,18-24,Une femme,Auvergne-Rhône-Alpes,de plus de 100 000 habitants (exemple : Grenob...,Vous exercez une activité professionnelle (act...,Professions intermédiaires administratives et ...,Professions Intermédiaires,"BTS, DUT, DEUST ou diplôme équivalent",1000 à 1500 euros,2,Non,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,France,Oui,NaN,#NULL!,Vous connaissez seulement le terme « essai cli...,"Non, pas du tout",Vous n&apos;avez pas d’avis,Vous n&apos;avez pas d’avis,Vous n&apos;avez pas d’avis,Vous n&apos;avez pas d’avis,Vous n&apos;avez pas d’avis,Vous n&apos;avez pas d’avis,Vous n&apos;avez pas d’avis,Vous n&apos;avez pas d’avis,Vous n&apos;avez pas d’avis,Vous n&apos;avez pas d’avis,Vous n&apos;avez pas d’avis,Vous n&apos;avez pas d’avis,Vous n&apos;avez pas d’avis,Vous n&apos;avez pas d’avis,Peu,A peu près autant de bien que de mal,"Non, vous pensez que les connaissances sont en...",Très bon,Non,Très facile,Très facile,Facile,Plutôt d’accord,"Ni d’accord, ni pas d’accord",Plutôt d’accord,Tout à fait d’accord,Tout à fait d’accord,Tout à fait d’accord,Plutôt d’accord,Tout à fait d’accord,Pas d’accord du tout,Vous n&apos;avez jamais entendu parler du CBD,#NULL!,#NULL!,Jamais,Moins d&apos;une fois par semaine,Jamais,Vous êtes déjà vacciné (une dose ou plus),Pas du tout favorable,Vous ne souhaitez pas répondre,"Oui, plutôt",Vous ne souhaitez pas répon

In [6]:
# Variable 'Q14'
# Question: Avez-vous déjà entendu parler d’essais cliniques ? 
labels_data['Q14'].value_counts()

Q14
Vous connaissez un peu les essais cliniques                                    840
Vous connaissez seulement le terme « essai clinique »                          722
Vous connaissez bien les essais cliniques                                      244
Vous n&apos;avez jamais entendu parler d’essais cliniques avant aujourd’hui    107
Vous connaissez très bien les essais cliniques                                  87
Vous ne souhaitez pas répondre                                                  22
Name: count, dtype: int64

In [7]:
labels_data['Q14'] = pd.Categorical(labels_data['Q14'], 
                            categories=['Vous n&apos;avez jamais entendu parler d’essais cliniques avant aujourd’hui'
                                        , 'Vous connaissez seulement le terme « essai clinique »'
                                        , 'Vous connaissez un peu les essais cliniques'
                                        , 'Vous connaissez bien les essais cliniques'
                                        , 'Vous connaissez très bien les essais cliniques'
                                        , 'Vous ne souhaitez pas répondre'], 
                            ordered=True)

In [8]:
list_indeps = ['Q02', 'Q07', 'Q01_R0', 'Q05_R0', 'Diplome', 'Q09', 'Q05_A', 'Q11', 'Q14', 'Q15', 'Q16_A', 'Q16_B'
              , 'Q16_C', 'Q16_D', 'Q16_E', 'Q16_F', 'Q17_A', 'Q17_B', 'Q17_C', 'Q17_D', 'Q17_E', 'Q17_F', 'Q17_G'
              , 'Q17_H', 'Ind_CI_Cat', 'Ind_NCI_Cat', 'Ind_CI_NSP_Cat', 'cluster_labels','Q18', 'Q19', 'Q20', 'Q21'
              , 'Q22', 'Q23_A', 'Q23_B', 'Q23_C', 'Q24_A', 'Q24_B', 'Q24_C', 'Q24_D', 'Q24_E', 'Q24_F', 'Q24_G', 'Q24_H'
              , 'Q24_I', 'Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34', 'Ty_hes_vaccinale', 'Q36_A', 'Q36_B', 'Q36_C', 'Q36_D'
              , 'Q36_E', 'Q36_F', 'Q36_G', 'Q36_H', 'Q36_I', 'Q36_J', 'Q36_K', 'Q36_L', 'Q37_A', 'Q37_B', 'Q37_C'
              , 'Q37_D', 'Q37_E', 'Q37_F', 'Q37_G', 'Q36_H', 'Q37_I', 'Q37_J', 'Q37_K', 'Q37_L', 'Q38M1', 'Q38M2', 'Q38M3'
              , 'Reseaux_sociaux_placement', 'Q39', 'Q40', 'Q41', 'competpol_score', 'Q42_A_cat', 'Q43_A', 'Q43_B', 'Q43_C'
              ,  'Q43_D', 'Q43_E', 'Q43_F', 'Q43_G', 'Q43_H', 'Q43_I', 'Q43_J']

questions_variables = {'Q02' : "Êtes-vous :", 'Q07' : "En prenant en compte tous les types de revenus que touchent les membres de votre foyer, quel est actuellement le montant mensuel net des ressources de l’ensemble votre ménage ?"
                      , 'Q01_R0': "Tranche d'âge", 'Q05_R0': "CSP", 'Diplome' : "À ce jour, le plus haut diplôme que vous possédez est… ?"
                      , 'Q09' : "Avez-vous des enfants à votre charge ?", 'Q05_A' : "Actuellement, quelle est votre situation ?"
                      , 'Q11' : "Dans quels pays êtes-vous né(e)?", 'Q14' : "Avez-vous déjà entendu parler d’essais cliniques ?"
                      , 'Q15' : " Sauriez-vous décrire ce qu’est un essai clinique randomisé ?"
                      , 'Q16_A' : "Veuillez indiquer dans quelle mesure vous êtes d'accord ou pas d'accord avec chacune de ces déclarations : Les industriels du médicament sont nécessaires pour la médecine moderne"
                      , 'Q16_B' : "Veuillez indiquer dans quelle mesure vous êtes d'accord ou pas d'accord avec chacune de ces déclarations : Les industriels du médicament sont assez encadrés par l’état"
                      , 'Q16_C' : "Veuillez indiquer dans quelle mesure vous êtes d'accord ou pas d'accord avec chacune de ces déclarations : Les industriels du médicament ont trop de pouvoir dans la société actuelle"
                      , 'Q16_D' : "Veuillez indiquer dans quelle mesure vous êtes d'accord ou pas d'accord avec chacune de ces déclarations : Les industriels du médicament pourraient être remplacés par des entreprises publiques"
                      , 'Q16_E' : "Veuillez indiquer dans quelle mesure vous êtes d'accord ou pas d'accord avec chacune de ces déclarations : Les industriels du médicament imposent leurs conditions aux Etats (prix des médicaments par exemple)"
                      , 'Q16_F' : "Veuillez indiquer dans quelle mesure vous êtes d'accord ou pas d'accord avec chacune de ces déclarations : Les industriels du médicament sont les principaux responsables des innovations sur les médicaments"
                      , 'Q17_A' : " Selon vous, est-ce que les situations suivantes sont un conflit d’intérêt pour un médecin quand il fait de la recherche : Etre payé personnellement pour écrire des articles dans la presse et intervenir sur les plateaux de télévision" 
                      , 'Q17_B' : "Selon vous, est-ce que les situations suivantes sont un conflit d’intérêt pour un médecin quand il fait de la recherche : Ecrire des articles dans la presse et intervenir sur les plateaux de télévision sans être payé personnellement"
                      , 'Q17_C' : " Selon vous, est-ce que les situations suivantes sont un conflit d’intérêt pour un médecin quand il fait de la recherche : Etre payé personnellement pour conseiller le gouvernement"
                      , 'Q17_D' : " Selon vous, est-ce que les situations suivantes sont un conflit d’intérêt pour un médecin quand il fait de la recherche : Conseiller le gouvernement sans être payé personnellement"
                      , 'Q17_E' : " Selon vous, est-ce que les situations suivantes sont un conflit d’intérêt pour un médecin quand il fait de la recherche : Être payé personnellement par un industriel pour réaliser des recherches"
                      , 'Q17_F' : " Selon vous, est-ce que les situations suivantes sont un conflit d’intérêt pour un médecin quand il fait de la recherche : Réaliser des recherches financées par un industriel sans être payé personnellement"
                      , 'Q17_G' : " Selon vous, est-ce que les situations suivantes sont un conflit d’intérêt pour un médecin quand il fait de la recherche : Être payé personnellement pour conseiller un industriel"
                      , 'Q17_H' : "Selon vous, est-ce que les situations suivantes sont un conflit d’intérêt pour un médecin quand il fait de la recherche : Conseiller un industriel sans être payé personnellement"
                      , 'Ind_CI_Cat' : "COI labelling propensity score", 'Ind_NCI_Cat' : "non-COI labelling propensity score "
                      , 'Ind_CI_NSP_Cat' : "COI uncertainty score", 'cluster_labels' : " Groups depending on their conception of what counts as a conflict of interest"
                      , 'Q18' : "Est-ce que vous vous intéressez à la science ?", 'Q19' : "D'une manière générale, avez-vous l'impression que la science  apporte à l'Homme plus de bien que de mal, plus de mal que de bien ou à peu près autant de bien que de mal ?"
                      , 'Q20' : "D’après vous, les connaissances actuelles permettent-elles de trancher le débat scientifique autour de l’efficacité de la chloroquine et de ses dérivés (comme l'hydroxychloroquine) contre la COVID-19 ?"
                      , 'Q21' : "Comment est votre état de santé en général ?", 'Q22' : "Avez-vous une maladie ou un problème de santé qui soit chronique ou de caractère durable ?"
                      , 'Q23_A' : "Pour vous, est-il facile ou difficile de payer toutes les factures à la fin du mois ?"
                      , 'Q23_B' : "Pour vous, est-il facile ou difficile d'acheter des médicaments si vous en avez besoin? (les médicaments prescrits qui ne sont pas ou que partiellement remboursés par l'assurance-maladie)"
                      , 'Q23_C' : "Payer les examens médicaux et les traitements si vous en avez besoin ? ( comme des traitements dentaires, des lunettes ou des traitements de médecine alternative, des mesures thérapeutiques ou des moyens auxiliaires)"
                      , 'Q24_A' : "Dans quelle mesure êtes-vous d’accord ou pas d’accord avec chacune des déclarations suivantes ? Les médecines alternatives/naturelles offrent de meilleures solutions aux problèmes de santé que la médecine conventionnelle"
                      , 'Q24_B' : "Dans quelle mesure êtes-vous d’accord ou pas d’accord avec chacune des déclarations suivantes ? Les médecines alternatives/naturelles promettent plus qu’elles ne peuvent offrir"
                      , 'Q24_C' : "Dans quelle mesure êtes-vous d’accord ou pas d’accord avec chacune des déclarations suivantes ? Il y a moins d’effets secondaires avec les médecines alternatives/naturelles."
                      , 'Q24_D' : "Dans quelle mesure êtes-vous d’accord ou pas d’accord avec chacune des déclarations suivantes ? Les médecines alternatives/naturelles, à base de plantes, sont plus saines que les médicaments prescrits par le médecin"
                      , 'Q24_E' : "Dans quelle mesure êtes-vous d’accord ou pas d’accord avec chacune des déclarations suivantes ? Les médecines alternatives/naturelles renforcent le corps et l’aident à se soigner par lui-même"
                      , 'Q24_F' : "Dans quelle mesure êtes-vous d’accord ou pas d’accord avec chacune des déclarations suivantes ? Pour être en bonne santé l’esprit est aussi important que le corps."
                      , 'Q24_G' : "Dans quelle mesure êtes-vous d’accord ou pas d’accord avec chacune des déclarations suivantes ? Une nourriture saine est plus efficace que les médicaments pour aider le corps à lutter contre les maladies"
                      , 'Q24_H' : "Dans quelle mesure êtes-vous d’accord ou pas d’accord avec chacune des déclarations suivantes ? Il existe des produits naturels meilleurs que les vaccins pour booster son système immunitaire"
                      , 'Q24_I' : "Dans quelle mesure êtes-vous d’accord ou pas d’accord avec chacune des déclarations suivantes ? Les médecines alternatives/naturelles ne doivent venir qu’en complément de la médecine conventionnelle"
                      , 'Q29' : "Comptez-vous vous faire vacciner contre le coronavirus à un moment ou un autre ?"
                      , 'Q30' : "Êtes-vous très, plutôt, plutôt pas ou pas du tout favorable aux vaccins en général ?", 'Q31' : "Êtes-vous favorable au vaccin contre la grippe ?"
                      , 'Q32' : "Êtes-vous favorable au vaccin contre les papillomavirus humains (HPV) ?", 'Q33' : "Êtes-vous favorable au vaccin contre la rougeole ?"
                      , 'Q34' : "Êtes-vous favorable au vaccin contre l’hépatite B ?", 'Ty_hes_vaccinale' : "Typologie hésitation vaccinale"
                      , 'Q36_A' : "Dans quelle mesure il est facile pour vous de comprendre le fonctionnement du système de santé ?"
                      , 'Q36_B' : "Dans quelle mesure il est facile pour vous de savoir à quel type de service médical ou prestataire de soins particulier vous adresser en cas de problème de santé ?"
                      , 'Q36_C' : "Dans quelle mesure il est facile pour vous de savoir dans quelle mesure votre assurance-maladie prend en charge les coûts de certaines prestations ?"
                      , 'Q36_D' : "Dans quelle mesure il est facile pour vous de comprendre les réformes en cours du système de santé qui pourraient affecter vos soins ?"
                      , 'Q36_E' : "Dans quelle mesure il est facile pour vous de vous renseigner sur vos droits en tant que patient ou utilisateur du système de santé ?"
                      , 'Q36_F' : "Dans quelle mesure il est facile pour vous de choisir un service médical ou un prestataire de soins en particulier ?"
                      , 'Q36_G' : "Dans quelle mesure il est facile pour vous de trouver des informations sur la qualité d'un service médical ou d'un prestataire de soins particulier ?"
                      , 'Q36_H' : "Dans quelle mesure il est facile pour vous de déterminer si un service médical ou un prestataire de soins particulier répondra à vos attentes et vos souhaits en matière de soins ?"
                      , 'Q36_I' : "Dans quelle mesure il est facile pour vous de comprendre comment prendre un rendez-vous auprès d'un service médical ou d'un prestataire de soins ?"
                      , 'Q36_J' : "Dans quelle mesure il est facile pour vous de vous renseigner sur les possibilités de soutien qui existent pour vous aider à vous orienter dans le système de santé ?"
                      , 'Q36_K' : "Dans quelle mesure il est facile pour vous de trouver le bon interlocuteur pour vos besoins au sein d'un établissement de santé ?"
                      , 'Q36_L' : "Dans quelle mesure il est facile pour vous de vous défendre si les soins que vous recevez ne répondent pas à vos besoins ?"
                      , 'Q37_A' : "Dans quelle mesure il est facile pour vous de savoir où obtenir de l'aide d'un professionnel de santé quand vous êtes malade ?"
                      , 'Q37_B' : "Dans quelle mesure il est facile pour vous de comprendre les informations sur ce qu'il faut faire en cas d'urgence médicale ?"
                      , 'Q37_C' : "Dans quelle mesure il est facile pour vous d'évaluer les risques et bénéfices des différents traitements médicaux possibles ?"
                      , 'Q37_D' : "Dans quelle mesure il est facile pour vous de suivre les consignes de votre médecin ou pharmacien ?"
                      , 'Q37_E' : "Dans quelle mesure il est facile pour vous de trouver des informations sur comment faire en cas de problèmes psychologiques ?"
                      , 'Q37_F' : "Dans quelle mesure il est facile pour vous de comprendre les informations sur les dépistages et examens recommandés ?"
                      , 'Q37_G' : "Dans quelle mesure il est facile pour vous d'évaluer la fiabilité des informations concernant l'impact sur la santé de certains comportements comme fumer, ne pas faire assez d'exercices ou trop boire d'alcool ?"
                      , 'Q37_H' : "Dans quelle mesure il est facile pour vous de savoir comment vous protéger des maladies à partir des informations disponibles dans les médias ?"
                      , 'Q37_I' : "Dans quelle mesure il est facile pour vous de trouver des informations sur les modes de vie sains, tels que l'activité physique ou une alimentation équilibrée ?" 
                      , 'Q37_J' : "Dans quelle mesure il est facile pour vous de comprendre les conseils de votre famille ou de vos amis en matière de santé ?"
                      , 'Q37_K' : "Dans quelle mesure il est facile pour vous d'évaluer comment vos conditions de logement peuvent affecter votre santé et votre bien-être ?"
                      , 'Q37_L' : "Dans quelle mesure il est facile pour vous de prendre des décisions pour améliorer votre santé et votre bien-être ?"
                      , 'Q38M1' : "Habituellement, pour vous informer, quel(s) moyen(s) utilisez-vous de préférence ? Item le plus important"
                      , 'Q38M2' : "Habituellement, pour vous informer, quel(s) moyen(s) utilisez-vous de préférence ? Second item le plus important"
                      , 'Q38M3' : "Habituellement, pour vous informer, quel(s) moyen(s) utilisez-vous de préférence ? Troisième item le plus important"
                      , 'Reseaux_sociaux_placement' : "Placement de réseaux sociaux dans les 3 premières sources d'information"
                      , 'Q39' : "Diriez-vous que vous vous intéressez à la politique ?", 'Q40' : "A quelle fréquence suivez-vous l’actualité politique ?"
                      , 'Q41' : "A quelle fréquence participez-vous aux élections ?", 'competpol_score' : "Compétence politique score"
                      , 'Q42_A_cat': "De laquelle des formations politiques suivantes vous sentez-vous le plus proche ou disons le moins éloigné ?"
                      , 'Q43_A' : "Avez-vous confiance ou pas confiance dans la justice ?", 'Q43_B' : "Avez-vous confiance ou pas confiance dans les médias ?"
                      , 'Q43_C' : "Avez-vous confiance ou pas confiance dans la police ?", 'Q43_D' : "Avez-vous confiance ou pas confiance dans le Parlement ?"
                      , 'Q43_E' : "Avez-vous confiance ou pas confiance dans la science ?", 'Q43_F' : "Avez-vous confiance ou pas confiance dans les grandes entreprises ?"
                      , 'Q43_G' : "Avez-vous confiance ou pas confiance dans les agences gouvernementales qui contrôlent les risques liés à la santé et à l'environnement ?"
                      , 'Q43_H' : "Avez-vous confiance ou pas confiance dans le gouvernement ?", 'Q43_I' : "Avez-vous confiance ou pas confiance dans les médecins ?"
                      , 'Q43_J' : "Avez-vous confiance ou pas confiance dans les industriels du médicament ?"}
                    

In [9]:
# Variable 'Q02'
# Question: Êtes-vous :
labels_data['Q02'].value_counts()

Q02
Une femme    1089
Un homme      932
Autre           1
Name: count, dtype: int64

In [10]:
labels_data['Q02'] = pd.Categorical(labels_data['Q02'], 
                            categories=['Une femme', 'Un homme'
                                        , 'Autre'], 
                            ordered=True)

In [11]:
# Variable 'Q07'
# Question : En prenant en compte tous les types de revenus que touchent les membres de votre foyer, quel est actuellement le montant mensuel net des ressources de l’ensemble votre ménage ?
labels_data['Q07'].value_counts()

Q07
2000 à 3000 euros                 465
3000 à 4000 euros                 337
1500 à 2000 euros                 306
1000 à 1500 euros                 253
4000 euros et plus                248
Vous ne souhaitez pas répondre    235
Moins de 1000 euros               178
Name: count, dtype: int64

In [12]:
#changer l'order des categories de la variable'Q07'
labels_data['Q07'] = pd.Categorical(labels_data['Q07'], 
                            categories=['Moins de 1000 euros', '1000 à 1500 euros' ,'1500 à 2000 euros'
                                        ,'2000 à 3000 euros', '3000 à 4000 euros'
                                       , '4000 euros et plus', 'Vous ne souhaitez pas répondre'], 
                            ordered=True)

In [13]:
# Variable 'Q01_R0'
# Question : Tranche d'âge
labels_data['Q01_R0'].value_counts()

Q01_R0
50-64     510
35-49     451
65-74     298
25-34     290
75plus    257
18-24     216
Name: count, dtype: int64

In [14]:
#changer l'order des categories de la variable 'Q01_R0'
labels_data['Q01_R0'] = pd.Categorical(labels_data['Q01_R0'], 
                            categories=['18-24', '25-34' ,'35-49'
                                        ,'50-64', '65-74'
                                       , '75plus'], 
                            ordered=True)

In [15]:
# Variable 'Q05_R0'
# Question : CSP
labels_data['Q05_R0'].value_counts()

Q05_R0
Retraités                                            678
Employés                                             333
Professions Intermédiaires                           291
Ouvriers                                             232
Cadres ou professions intellectuelles supérieures    225
Autres personnes sans activité professionnelle       172
Artisans, commerçants ou chefs d&apos;entreprise      76
Agriculteurs                                          15
Name: count, dtype: int64

In [16]:
# Variable 'Q06'
# Question : À ce jour, le plus haut diplôme que vous possédez est… ?
labels_data['Q06'].value_counts()

Q06
CAP, BEP, mention complémentaire, diplôme de la santé ou du travail social de niveau équivalent                        406
BTS, DUT, DEUST ou diplôme équivalent                                                                                  266
Baccalauréat technologique ou professionnel                                                                            245
Diplôme de 2ème cycle universitaire (licence, L3, maîtrise, M1) ou diplôme équivalent (y/c CAPES)                      209
Baccalauréat général, brevet supérieur, capacité en droit, DAEU ou diplôme équivalent                                  174
BEPC, brevet des collèges, brevet élémentaire, 1ère partie du baccalauréat                                             166
Diplôme de 3ème cycle universitaire (DES, DEA, DESS, Master, M2) y compris médecine, pharmacie, dentaire ou diplôme    127
Brevet professionnel ou de technicien, brevet des métiers, BEA, BEI, BEC, BEH, BES ou diplôme équivalent                99
Diplôme d’un

In [17]:
labels_data['Diplome'] = labels_data['Q06'].apply(lambda x : x.replace("Aucun diplôme", "Aucun diplôme, CEP").
                                        replace("Certificat d’études primaires, diplôme de fin d’études obligatoires","Aucun diplôme, CEP").
                                        replace("BEPC, brevet des collèges, brevet élémentaire, 1ère partie du baccalauréat","Avant Bac").
                                        replace("CAP, BEP, mention complémentaire, diplôme de la santé ou du travail social de niveau équivalent","Avant Bac").
                                        replace("Brevet professionnel ou de technicien, brevet des métiers, BEA, BEI, BEC, BEH, BES ou diplôme équivalent", "Avant Bac").
                                        replace("Baccalauréat technologique ou professionnel","Bac techno pro" ).
                                        replace("Baccalauréat général, brevet supérieur, capacité en droit, DAEU ou diplôme équivalent", "Bac").
                                        replace("BTS, DUT, DEUST ou diplôme équivalent", "BTS DUT").
                                        replace("Diplôme de la santé et du travailleur social (infirmière, puéricultrice, éducateur...)", "Diplôme santé").
                                        replace("Diplôme de 2ème cycle universitaire (licence, L3, maîtrise, M1) ou diplôme équivalent (y/c CAPES)", "Licence").
                                        replace("Diplôme de 3ème cycle universitaire (DES, DEA, DESS, Master, M2) y compris médecine, pharmacie, dentaire ou diplôme", "Master ou plus").
                                        replace("Diplôme d’une grande école (ingénieur, commerce...) ou diplôme équivalent", "Master ou plus").
                                        replace("Diplôme de 1er cycle universitaire (DEUG, L1, L2) ou diplôme équivalent", "Licence").
                                        replace("Doctorat de 3ème cycle universitaire", "Master ou plus")
                                        )

In [18]:
labels_data['Diplome'].value_counts()

Diplome
Avant Bac             671
BTS DUT               266
Licence               256
Bac techno pro        245
Master ou plus        232
Bac                   174
Aucun diplôme, CEP    122
Diplôme santé          56
Name: count, dtype: int64

In [19]:
#changer l'order des categories de la variable 'Diplome'
labels_data['Diplome'] = pd.Categorical(labels_data['Diplome'], 
                            categories=['Aucun diplôme, CEP', 'Avant Bac' ,'Bac techno pro'
                                        ,'Bac', 'BTS DUT', 'Diplôme santé'
                                       , 'Licence', 'Master ou plus'], 
                            ordered=True)

In [20]:
# Variable 'Q09'
# Question : Avez-vous des enfants à votre charge ?
labels_data['Q09'].value_counts()

Q09
Non                                                                                         1380
Oui                                                                                          623
Vous n’avez pas d’enfant actuellement mais vous en attendez un, ou essayez d’en avoir un      19
Name: count, dtype: int64

In [21]:
# Variable 'Q05_A'
# Question: Actuellement, quelle est votre situation ?
labels_data['Q05_A'].value_counts()

Q05_A
Vous exercez une activité professionnelle (actifs, apprentis, stagiaires)          893
Vous êtes à la retraite ou en pré-retraite                                         678
Vous êtes homme ou femme au foyer                                                  137
Vous êtes dans une autre situation (invalide, sans activité professionnelle...)    107
Vous êtes chômeur ayant déjà travaillé                                              98
Vous êtes collégien, lycéen ou étudiant                                             84
Vous êtes à la recherche d&apos;un premier emploi                                   25
Name: count, dtype: int64

In [22]:
labels_data['Q05_A'] = labels_data['Q05_A'].astype('category')

In [23]:
# Variable 'Q11'
# Question: Dans quels pays êtes-vous né(e)?
labels_data['Q11'].value_counts()

Q11
France        1908
Autre pays     114
Name: count, dtype: int64

In [24]:
# Variable 'Q14'
# Question: Avez-vous déjà entendu parler d’essais cliniques ?
labels_data['Q14'].value_counts()

Q14
Vous connaissez un peu les essais cliniques                                    840
Vous connaissez seulement le terme « essai clinique »                          722
Vous connaissez bien les essais cliniques                                      244
Vous n&apos;avez jamais entendu parler d’essais cliniques avant aujourd’hui    107
Vous connaissez très bien les essais cliniques                                  87
Vous ne souhaitez pas répondre                                                  22
Name: count, dtype: int64

In [25]:
#changer l'order des categories de la variable 'Q14'
labels_data['Q14'] = pd.Categorical(labels_data['Q14'], 
                            categories=['Vous n&apos;avez jamais entendu parler d’essais cliniques avant aujourd’hui'
                                        , 'Vous connaissez seulement le terme « essai clinique »' 
                                        , 'Vous connaissez un peu les essais cliniques'
                                        , 'Vous connaissez bien les essais cliniques'
                                        , 'Vous connaissez très bien les essais cliniques'
                                        , 'Vous ne souhaitez pas répondre'], 
                            ordered=True)

In [26]:
# Variable 'Q15'
# Question: Sauriez-vous décrire ce qu’est un essai clinique randomisé ?
labels_data['Q15'].value_counts()

Q15
Non, pas du tout                  703
Oui, plus ou moins                541
Non, pas vraiment                 532
#NULL!                            129
Oui, assez précisément            109
Vous ne souhaitez pas répondre      8
Name: count, dtype: int64

In [27]:
#changer l'order des categories de la variable 'Q15'
labels_data['Q15'] = pd.Categorical(labels_data['Q15'], 
                            categories=['#NULL!', 'Non, pas du tout'
                                        , 'Non, pas vraiment' 
                                        , 'Oui, plus ou moins'
                                        , 'Oui, assez précisément'
                                        , 'Vous ne souhaitez pas répondre'], 
                            ordered=True)

In [28]:
# Variable 'Q16_A'
# Question: Veuillez indiquer dans quelle mesure vous êtes d'accord ou pas d'accord avec chacune de ces déclarations : Les industriels du médicament sont nécessaires pour la médecine moderne
labels_data['Q16_A'].value_counts()

Q16_A
Plutôt d’accord                   1099
Tout à fait d’accord               618
Plutôt pas d’accord                139
Vous n&apos;avez pas d’avis        109
Pas d’accord du tout                42
Vous ne souhaitez pas répondre      15
Name: count, dtype: int64

In [29]:
# Variable 'Q16_B'
# Question: Veuillez indiquer dans quelle mesure vous êtes d'accord ou pas d'accord avec chacune de ces déclarations : Les industriels du médicament sont assez encadrés par l’état 
labels_data['Q16_B'].value_counts()

Q16_B
Plutôt d’accord                   770
Plutôt pas d’accord               584
Pas d’accord du tout              257
Tout à fait d’accord              210
Vous n&apos;avez pas d’avis       179
Vous ne souhaitez pas répondre     22
Name: count, dtype: int64

In [30]:
# Variable 'Q16_C'
# Question: Veuillez indiquer dans quelle mesure vous êtes d'accord ou pas d'accord avec chacune de ces déclarations : Les industriels du médicament ont trop de pouvoir dans la société actuelle 
labels_data['Q16_C'].value_counts()

Q16_C
Plutôt d’accord                   822
Tout à fait d’accord              596
Plutôt pas d’accord               345
Vous n&apos;avez pas d’avis       173
Pas d’accord du tout               69
Vous ne souhaitez pas répondre     17
Name: count, dtype: int64

In [31]:
# Variable 'Q16_D'
# Question: Veuillez indiquer dans quelle mesure vous êtes d'accord ou pas d'accord avec chacune de ces déclarations : Les industriels du médicament pourraient être remplacés par des entreprises publiques 
labels_data['Q16_D'].value_counts()

Q16_D
Plutôt d’accord                   674
Plutôt pas d’accord               444
Tout à fait d’accord              372
Vous n&apos;avez pas d’avis       290
Pas d’accord du tout              218
Vous ne souhaitez pas répondre     24
Name: count, dtype: int64

In [32]:
# Variable 'Q16_E'
# Question: Veuillez indiquer dans quelle mesure vous êtes d'accord ou pas d'accord avec chacune de ces déclarations : Les industriels du médicament imposent leurs conditions aux Etats (prix des médicaments par exemple)
labels_data['Q16_E'].value_counts()

Q16_E
Plutôt d’accord                   828
Tout à fait d’accord              726
Vous n&apos;avez pas d’avis       203
Plutôt pas d’accord               189
Pas d’accord du tout               54
Vous ne souhaitez pas répondre     22
Name: count, dtype: int64

In [33]:
# Variable 'Q16_F'
# Question: Veuillez indiquer dans quelle mesure vous êtes d'accord ou pas d'accord avec chacune de ces déclarations : Les industriels du médicament sont les principaux responsables des innovations sur les médicaments
labels_data['Q16_F'].value_counts()

Q16_F
Plutôt d’accord                   1014
Tout à fait d’accord               415
Plutôt pas d’accord                292
Vous n&apos;avez pas d’avis        204
Pas d’accord du tout                82
Vous ne souhaitez pas répondre      15
Name: count, dtype: int64

In [34]:
Items16 = ['Q16_A', 'Q16_B', 'Q16_C', 'Q16_D', 'Q16_E', 'Q16_F']

In [35]:
for i in labels_data[Items16].columns:
    labels_data[i] = pd.Categorical(labels_data[i], 
                            categories=['Pas d’accord du tout', 'Plutôt pas d’accord'
                                        ,'Plutôt d’accord', 'Tout à fait d’accord'
                                        , 'Vous n&apos;avez pas d’avis'
                                        , 'Vous ne souhaitez pas répondre'] 
                                        , ordered=True)

In [36]:
for i in labels_data[Items16].columns:
    labels_data[i] =labels_data[i].cat.rename_categories({'Pas d’accord du tout': "Pas d’accord du tout"
                                                              , 'Plutôt pas d’accord': "Plutôt pas d’accord"
                                                              , 'Plutôt d’accord' : "Plutôt d’accord"
                                                              , 'Tout à fait d’accord' : "Tout à fait d’accord"
                                                              , 'Vous n&apos;avez pas d’avis' : "Vous n’avez pas d’avis"
                                                              , 'Vous ne souhaitez pas répondre' : "Vous ne souhaitez pas répondre"})

In [37]:
# Variable 'Q17_A'
# Question:  Selon vous, est-ce que les situations suivantes sont un conflit d’intérêt pour un médecin quand il fait de la recherche : Etre payé personnellement pour écrire des articles dans la presse et intervenir sur les plateaux de télévision
labels_data['Q17_A'].value_counts()

Q17_A
Plutôt                            726
Tout à fait                       549
Plutôt pas                        325
Vous n&apos;avez pas d’avis       257
Pas du tout                       133
Vous ne souhaitez pas répondre     32
Name: count, dtype: int64

In [38]:
# Variable 'Q17_B'
# Question: Selon vous, est-ce que les situations suivantes sont un conflit d’intérêt pour un médecin quand il fait de la recherche : Ecrire des articles dans la presse et intervenir sur les plateaux de télévision sans être payé personnellement
labels_data['Q17_B'].value_counts()

Q17_B
Plutôt pas                        600
Plutôt                            507
Pas du tout                       351
Vous n&apos;avez pas d’avis       268
Tout à fait                       258
Vous ne souhaitez pas répondre     38
Name: count, dtype: int64

In [39]:
# Variable 'Q17_C'
# Question:  Selon vous, est-ce que les situations suivantes sont un conflit d’intérêt pour un médecin quand il fait de la recherche : Etre payé personnellement pour conseiller le gouvernement 
labels_data['Q17_C'].value_counts()

Q17_C
Plutôt                            648
Tout à fait                       460
Plutôt pas                        434
Vous n&apos;avez pas d’avis       251
Pas du tout                       186
Vous ne souhaitez pas répondre     43
Name: count, dtype: int64

In [40]:
# Variable 'Q17_D'
# Question:  Selon vous, est-ce que les situations suivantes sont un conflit d’intérêt pour un médecin quand il fait de la recherche : Conseiller le gouvernement sans être payé personnellement
labels_data['Q17_D'].value_counts()

Q17_D
Plutôt pas                        619
Plutôt                            466
Pas du tout                       410
Vous n&apos;avez pas d’avis       261
Tout à fait                       227
Vous ne souhaitez pas répondre     39
Name: count, dtype: int64

In [41]:
# Variable 'Q17_E'
# Question:  Selon vous, est-ce que les situations suivantes sont un conflit d’intérêt pour un médecin quand il fait de la recherche : Être payé personnellement par un industriel pour réaliser des recherches
labels_data['Q17_E'].value_counts()

Q17_E
Plutôt                            643
Tout à fait                       551
Plutôt pas                        356
Vous n&apos;avez pas d’avis       228
Pas du tout                       197
Vous ne souhaitez pas répondre     47
Name: count, dtype: int64

In [42]:
# Variable 'Q17_F'
# Question:  Selon vous, est-ce que les situations suivantes sont un conflit d’intérêt pour un médecin quand il fait de la recherche : Réaliser des recherches financées par un industriel sans être payé personnellement
labels_data['Q17_F'].value_counts()

Q17_F
Plutôt pas                        613
Plutôt                            455
Pas du tout                       393
Vous n&apos;avez pas d’avis       279
Tout à fait                       235
Vous ne souhaitez pas répondre     47
Name: count, dtype: int64

In [43]:
# Variable 'Q17_G'
# Question:  Selon vous, est-ce que les situations suivantes sont un conflit d’intérêt pour un médecin quand il fait de la recherche : Être payé personnellement pour conseiller un industriel
labels_data['Q17_G'].value_counts()

Q17_G
Plutôt                            697
Tout à fait                       455
Plutôt pas                        387
Vous n&apos;avez pas d’avis       260
Pas du tout                       183
Vous ne souhaitez pas répondre     40
Name: count, dtype: int64

In [44]:
# Variable 'Q17_H'
# Question: Selon vous, est-ce que les situations suivantes sont un conflit d’intérêt pour un médecin quand il fait de la recherche : Conseiller un industriel sans être payé personnellement
labels_data['Q17_H'].value_counts()

Q17_H
Plutôt pas                        664
Plutôt                            443
Pas du tout                       371
Vous n&apos;avez pas d’avis       288
Tout à fait                       202
Vous ne souhaitez pas répondre     54
Name: count, dtype: int64

In [45]:
Items17 = ['Q17_A', 'Q17_B' , 'Q17_C', 'Q17_D', 'Q17_E', 'Q17_F', 'Q17_G', 'Q17_H']

In [46]:
for i in labels_data[Items17].columns:
    labels_data[i] = pd.Categorical(labels_data[i], 
                            categories=['Pas du tout', 'Plutôt pas'
                                        ,'Plutôt', 'Tout à fait'
                                        , 'Vous n&apos;avez pas d’avis'
                                        , 'Vous ne souhaitez pas répondre'] 
                                        , ordered=True)

In [47]:
for i in labels_data[Items17].columns:
    labels_data[i] =labels_data[i].cat.rename_categories({'Pas du tout': "Pas du tout"
                                                              , 'Plutôt pas': "Plutôt pas"
                                                              , 'Plutôt' : "Plutôt"
                                                              , 'Tout à fait' : "Tout à fait"
                                                              , 'Vous n&apos;avez pas d’avis' : "Vous n’avez pas d’avis"
                                                              , 'Vous ne souhaitez pas répondre' : "Vous ne souhaitez pas répondre"})

In [48]:
labels_data['Q17_H'].value_counts()

Q17_H
Plutôt pas                        664
Plutôt                            443
Pas du tout                       371
Vous n’avez pas d’avis            288
Tout à fait                       202
Vous ne souhaitez pas répondre     54
Name: count, dtype: int64

In [49]:
# construire le premier indicateur d'attitudes à l'égard des conflits d'interet CI:
# définir une fonction pour recoder les variables des items 17 :
def Conflit_interet_Q17(x):
    if x == "Plutôt":
        return 1
    if x == "Tout à fait":
        return 1
    if x == "Plutôt pas":
        return 0
    if x == "Pas du tout":
        return 0
    if x == "Vous n'avez pas d’avis" or x == "Vous ne souhaitez pas répondre" :
        return 0

In [50]:
labels_data['Q17_A_cat']= labels_data['Q17_A'].apply(lambda x : Conflit_interet_Q17(x))
labels_data['Q17_B_cat']= labels_data['Q17_B'].apply(lambda x : Conflit_interet_Q17(x))
labels_data['Q17_C_cat']= labels_data['Q17_C'].apply(lambda x : Conflit_interet_Q17(x))
labels_data['Q17_D_cat']= labels_data['Q17_D'].apply(lambda x : Conflit_interet_Q17(x))
labels_data['Q17_E_cat']= labels_data['Q17_E'].apply(lambda x : Conflit_interet_Q17(x))
labels_data['Q17_F_cat']= labels_data['Q17_F'].apply(lambda x : Conflit_interet_Q17(x))
labels_data['Q17_G_cat']= labels_data['Q17_G'].apply(lambda x : Conflit_interet_Q17(x))
labels_data['Q17_H_cat']= labels_data['Q17_H'].apply(lambda x : Conflit_interet_Q17(x))

In [51]:
labels_data['Ind_CI'] = labels_data[['Q17_A_cat', 'Q17_B_cat', 'Q17_C_cat', 'Q17_D_cat'
                                    , 'Q17_E_cat', 'Q17_F_cat', 'Q17_G_cat', 'Q17_H_cat']].sum(axis=1)

In [52]:
labels_data['Ind_CI'].value_counts()

Ind_CI
4.0    386
0.0    303
3.0    287
8.0    221
5.0    216
2.0    194
6.0    190
1.0    139
7.0     86
Name: count, dtype: int64

In [53]:
# construire le deuxieme indicateur d'attitudes à l'égard des conflits d'interet NCI:
# définir une fonction pour recoder les variables des items 17 :
def NON_Conflit_interet_Q17(x):
    if x == "Plutôt":
        return 0
    if x == "Tout à fait":
        return 0
    if x == "Plutôt pas":
        return 1
    if x == "Pas du tout":
        return 1
    if x == "Vous n'avez pas d’avis" or x == "Vous ne souhaitez pas répondre" :
        return 1

In [54]:
labels_data['Q17_A_cat_No']= labels_data['Q17_A'].apply(lambda x : NON_Conflit_interet_Q17(x))
labels_data['Q17_B_cat_No']= labels_data['Q17_B'].apply(lambda x : NON_Conflit_interet_Q17(x))
labels_data['Q17_C_cat_No']= labels_data['Q17_C'].apply(lambda x : NON_Conflit_interet_Q17(x))
labels_data['Q17_D_cat_No']= labels_data['Q17_D'].apply(lambda x : NON_Conflit_interet_Q17(x))
labels_data['Q17_E_cat_No']= labels_data['Q17_E'].apply(lambda x : NON_Conflit_interet_Q17(x))
labels_data['Q17_F_cat_No']= labels_data['Q17_F'].apply(lambda x : NON_Conflit_interet_Q17(x))
labels_data['Q17_G_cat_No']= labels_data['Q17_G'].apply(lambda x : NON_Conflit_interet_Q17(x))
labels_data['Q17_H_cat_No']= labels_data['Q17_H'].apply(lambda x : NON_Conflit_interet_Q17(x))

In [55]:
labels_data['Ind_NCI'] = labels_data[['Q17_A_cat_No', 'Q17_B_cat_No', 'Q17_C_cat_No', 'Q17_D_cat_No'
                                    , 'Q17_E_cat_No', 'Q17_F_cat_No', 'Q17_G_cat_No', 'Q17_H_cat_No']].sum(axis=1)

In [56]:
labels_data['Ind_NCI'].value_counts()

Ind_NCI
0.0    409
4.0    382
3.0    258
2.0    242
5.0    231
6.0    149
1.0    142
8.0    122
7.0     87
Name: count, dtype: int64

In [57]:
# construire le troisieme indicateur d'attitudes à l'égard des conflits d'interet CI_NSP:
# définir une fonction pour recoder les variables des items 17 :
def NSP_Conflit_interet_Q17(x):
    if x == "Plutôt":
        return 0
    if x == "Tout à fait":
        return 0
    if x == "Plutôt pas":
        return 0
    if x == "Pas du tout":
        return 0
    if x == "Vous n'avez pas d’avis" or x == "Vous ne souhaitez pas répondre" :
        return 1

In [58]:
labels_data['Q17_A_cat_NSP']= labels_data['Q17_A'].apply(lambda x : NSP_Conflit_interet_Q17(x))
labels_data['Q17_B_cat_NSP']= labels_data['Q17_B'].apply(lambda x : NSP_Conflit_interet_Q17(x))
labels_data['Q17_C_cat_NSP']= labels_data['Q17_C'].apply(lambda x : NSP_Conflit_interet_Q17(x))
labels_data['Q17_D_cat_NSP']= labels_data['Q17_D'].apply(lambda x : NSP_Conflit_interet_Q17(x))
labels_data['Q17_E_cat_NSP']= labels_data['Q17_E'].apply(lambda x : NSP_Conflit_interet_Q17(x))
labels_data['Q17_F_cat_NSP']= labels_data['Q17_F'].apply(lambda x : NSP_Conflit_interet_Q17(x))
labels_data['Q17_G_cat_NSP']= labels_data['Q17_G'].apply(lambda x : NSP_Conflit_interet_Q17(x))
labels_data['Q17_H_cat_NSP']= labels_data['Q17_H'].apply(lambda x : NSP_Conflit_interet_Q17(x))

In [59]:
labels_data['Ind_CI_NSP'] = labels_data[['Q17_A_cat_NSP', 'Q17_B_cat_NSP', 'Q17_C_cat_NSP', 'Q17_D_cat_NSP'
                                    , 'Q17_E_cat_NSP', 'Q17_F_cat_NSP', 'Q17_G_cat_NSP', 'Q17_H_cat_NSP']].sum(axis=1)

In [60]:
labels_data['Ind_CI_NSP'].value_counts()

Ind_CI_NSP
0.0    1869
1.0      90
2.0      29
8.0      13
3.0       9
4.0       4
5.0       4
6.0       3
7.0       1
Name: count, dtype: int64

In [61]:
# définir une fonction pour grouper les valeurs des indicateurs 
def reco_category(x):
    if x>=7 :
        return "7-8"
    if x>=6 :
        return "5-6"
    if x>=4:
        return "3-4"
    if x>0:
        return "1-2"
    return 0
labels_data['Ind_CI_Cat'] = labels_data['Ind_CI'].apply(reco_category)
labels_data['Ind_NCI_Cat'] = labels_data['Ind_NCI'].apply(reco_category)
labels_data['Ind_CI_NSP_Cat'] = labels_data['Ind_CI_NSP'].apply(reco_category)

In [62]:
labels_data['Ind_CI_Cat'].value_counts()

Ind_CI_Cat
1-2    620
3-4    602
7-8    307
0      303
5-6    190
Name: count, dtype: int64

In [63]:
scores_CI = ['Ind_CI_Cat', 'Ind_NCI_Cat', 'Ind_CI_NSP_Cat']

In [64]:
for i in labels_data[scores_CI].columns:
    labels_data[i] = pd.Categorical(labels_data[i], 
                            categories=[0, '1-2'
                                        ,'3-4', '5-6'
                                        , '7-8'] 
                                        , ordered=True)

In [65]:
# construire une typologie de conflits d'interets:
# définir une fonction pour recoder les variables des items 17 :
def CI_codage_clustering(x):
    if x == "Plutôt":
        return "D’accord"
    if x == "Tout à fait":
        return "D’accord"
    if x == "Plutôt pas":
        return "Pas d’accord"
    if x == "Pas du tout":
        return "Pas d’accord"
    if x == "Vous n’avez pas d’avis" or x == "Vous ne souhaitez pas répondre" :
        return "Je ne sais pas"

In [66]:
labels_data['Q17_A_cat_clustering']= labels_data['Q17_A'].apply(lambda x : CI_codage_clustering(x))
labels_data['Q17_B_cat_clustering']= labels_data['Q17_B'].apply(lambda x : CI_codage_clustering(x))
labels_data['Q17_C_cat_clustering']= labels_data['Q17_C'].apply(lambda x : CI_codage_clustering(x))
labels_data['Q17_D_cat_clustering']= labels_data['Q17_D'].apply(lambda x : CI_codage_clustering(x))
labels_data['Q17_E_cat_clustering']= labels_data['Q17_E'].apply(lambda x : CI_codage_clustering(x))
labels_data['Q17_F_cat_clustering']= labels_data['Q17_F'].apply(lambda x : CI_codage_clustering(x))
labels_data['Q17_G_cat_clustering']= labels_data['Q17_G'].apply(lambda x : CI_codage_clustering(x))
labels_data['Q17_H_cat_clustering']= labels_data['Q17_H'].apply(lambda x : CI_codage_clustering(x))

In [67]:
labels_data['Q17_H_cat_clustering'].value_counts()

Q17_H_cat_clustering
Pas d’accord      1035
D’accord           645
Je ne sais pas     342
Name: count, dtype: int64

In [68]:
# HCPC 3 clusters 
var_fac = ["Q17_A_cat_clustering","Q17_B_cat_clustering","Q17_C_cat_clustering","Q17_D_cat_clustering",
      "Q17_E_cat_clustering","Q17_F_cat_clustering",
      "Q17_G_cat_clustering",
      "Q17_H_cat_clustering"]
X2 = labels_data[var_fac]
mca = prince.MCA(n_components=3)
mca = mca.fit(X2)
X2t = mca.transform(X2)
cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')
labels_data["cat_CI2"] = cluster.fit_predict(X2t)
labels_data["cluster_labels"] = labels_data["cat_CI2"].replace({0:"Money matters",1:"Uncertain",2:"Ivory tower"})

In [69]:
labels_data

,CaseId,STATUT,poids_init,poids_redresses_stand,INT01,Q01,Q01_R0,Q02,Q03,Q04,Q05_A,Q05_B,Q05_R0,Q06,Q07,Q08,Q09,Q10_A,Q10_B,Q10_C,Q10_D,Q10_E,Q11,Q11_1,Q12,Q13,Q14,Q15,Q16_A,Q16_B,Q16_C,Q16_D,Q16_E,Q16_F,Q17_A,Q17_B,Q17_C,Q17_D,Q17_E,Q17_F,Q17_G,Q17_H,Q18,Q19,Q20,Q21,Q22,Q23_A,Q23_B,Q23_C,Q24_A,Q24_B,Q24_C,Q24_D,Q24_E,Q24_F,Q24_G,Q24_H,Q24_I,Q25,Q26,Q27,Q28_A,Q28_B,Q28_C,Q29,Q44,Q30,Q31,Q32,Q33,Q34,Q29_1,Q29_2,Q29_2_PRE,Q29_3,Q35,Q36_A,Q36_B,Q36_C,Q36_D,Q36_E,Q36_F,Q36_G,Q36_H,Q36_I,Q36_J,Q36_K,Q36_L,Q37_A,Q37_B,Q37_C,Q37_D,Q37_E,Q37_F,Q37_G,Q37_H,Q37_I,Q37_J,Q37_K,Q37_L,Q38M1,Q38M2,Q38M3,Q39,Q40,Q41,Q42_A,Q42_B,Q43_A,Q43_B,Q43_C,Q43_D,Q43_E,Q43_F,Q43_G,Q43_H,Q43_I,Q43_J,COMMENTAIRES,Diplome,Q17_A_cat,Q17_B_cat,Q17_C_cat,Q17_D_cat,Q17_E_cat,Q17_F_cat,Q17_G_cat,Q17_H_cat,Ind_CI,Q17_A_cat_No,Q17_B_cat_No,Q17_C_cat_No,Q17_D_cat_No,Q17_E_cat_No,Q17_F_cat_No,Q17_G_cat_No,Q17_H_cat_No,Ind_NCI,Q17_A_cat_NSP,Q17_B_cat_NSP,Q17_C_cat_NSP,Q17_D_cat_NSP,Q17_E_cat_NSP,Q17_F_cat_NSP,Q17_G_cat_NSP,Q17_H_cat_NSP,Ind_CI_NSP,Ind_CI_Cat,Ind_NCI_Cat,Ind_CI_NSP_Cat,Q17_A_cat_clustering,Q17_B_cat_clustering,Q17_C_cat_clustering,Q17_D_cat_clustering,Q17_E_cat_clustering,Q17_F_cat_clustering,Q17_G_cat_clustering,Q17_H_cat_clustering,cat_CI2,cluster_labels
0,10,Complet,25416.857072,1.015050,Oui,1946,65-74,Un homme,Provence-Alpes-Côte d&apos;Azur,de 2 000 à moins de 20 000 habitants (exemple ...,Vous êtes à la retraite ou en pré-retraite,Ingénieurs et cadres techniques d&apos;entreprise,Retraités,"Diplôme d’une grande école (ingénieur, commerc...",4000 euros et plus,2,Non,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,France,Oui,NaN,#NULL!,Vous connaissez un peu les essais cliniques,"Oui, plus ou moins",Tout à fait d’accord,Plutôt d’accord,Plutôt d’accord,Plutôt pas d’accord,Tout à fait d’accord,Plutôt pas d’accord,Plutôt,Plutôt,Tout à fait,Pas du tout,Plutôt,Pas du tout,Plutôt,Plutôt pas,Assez,A peu près autant de bien que de mal,"Oui, et vous pensez que la chloroquine et ses ...",Assez bon,"Oui, j’ai un problème de santé",Très facile,Facile,Facile,Tout à fait d’accord,"Ni d’accord, ni pas d’accord",Plutôt d’accord,Plutôt d’accord,Plutôt d’accord,Tout à fait d’accord,Plutôt d’accord,Plutôt d’accord,Plutôt d’accord,Vous connaissez seulement le terme “CBD”,Un peu,Jamais,Jamais,Tous les jours ou presque,Jamais,Vous êtes déjà vacciné (une dose ou plus),Plutôt pas favorable,Plutôt favorable,"Oui, tout à fait","Oui, plutôt","Oui, tout à fait","Oui, tout à fait",Mars 2021,Non,NaN,#NULL!,"Oui, mais vous l&apos;avez désinstallée",Difficile,Facile,Difficile,Très difficile,Difficile,Difficile,Difficile,Difficile,Facile,Difficile,Difficile,Difficile,Facile,Facile,Difficile,Facile,Difficile,Facile,Difficile,Difficile,Facile,Facile,Difficile,Difficile,Les échanges avec vos proches,La presse écrite imprimée,"Les réseaux sociaux (Facebook, Twitter, etc.)","Oui, beaucoup",Tous les jours ou presque,Vous votez à toutes les élections,Les Républicains,#NULL!,Plutôt pas confiance,Pas du tout confiance,Plutôt confiance,Plutôt pas confiance,Plutôt confiance,Plutôt confiance,Pas du tout confiance,Pas du tout confiance,Plutôt confiance,Plutôt confiance,NaN,Master ou plus,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,5.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3-4,1-2,0,D’accord,D’accord,D’accord,Pas d’accord,D’accord,Pas d’accord,D’accord,Pas d’accord,0,Money matters
1,1000,Complet,25416.857072,0.678083,Oui,2000,18-24,Une femme,Auvergne-Rhône-Alpes,de plus de 100 000 habitants (exemple : Grenob...,Vous exercez une activité professionnelle (act...,Professions intermédiaires administratives et ...,Professions Intermédiaires,"BTS, DUT, DEUST ou diplôme équivalent",1000 à 1500 euros,2,Non,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,France,Oui,NaN,#NULL!,Vous connaissez seulement le terme « essai cli...,"Non, pas du tout",Vous n’avez pas d’avis,Vous n’avez pas d’avis,Vous n’avez pas d’avis,Vous n’avez pas d’avis,Vous n’avez pas d’avis,Vous n’avez pas d’avis,Vous n’avez pas d’avis,Vous

In [70]:
#changer l'order des categories de la variable 'cluster_labels'
labels_data['cluster_labels'] = pd.Categorical(labels_data['cluster_labels'], 
                            categories=['Money matters', 'Ivory tower'
                                        ,'Uncertain'], 
                            ordered=True)

In [71]:
# Variable 'Q18'
# Question: Est-ce que vous vous intéressez à la science ?
labels_data['Q18'].value_counts()

Q18
Assez          921
Peu            677
Beaucoup       283
Pas du tout    141
Name: count, dtype: int64

In [72]:
#changer l'order des categories de la variable 'Q18'
labels_data['Q18'] = pd.Categorical(labels_data['Q18'], 
                            categories=['Pas du tout', 'Peu'
                                        ,'Assez', 'Beaucoup'],
                            ordered=True)

In [73]:
# Variable 'Q19'
# Question: D'une manière générale, avez-vous l'impression que la science apporte à l'Homme plus de bien que de mal, plus de mal que de bien ou à peu près autant de bien que de mal ?
labels_data['Q19'].value_counts()

Q19
A peu près autant de bien que de mal    860
Plus de bien que de mal                 836
Plus de mal que de bien                 208
Vous n&apos;avez pas d’avis             106
Vous ne souhaitez pas répondre           12
Name: count, dtype: int64

In [74]:
#changer l'order des categories de la variable 'Q19'
labels_data['Q19'] = pd.Categorical(labels_data['Q19'], 
                            categories=['Plus de mal que de bien', 'A peu près autant de bien que de mal'
                                        ,'Plus de bien que de mal', 'Vous n&apos;avez pas d’avis'
                                       , 'Vous ne souhaitez pas répondre'],
                            ordered=True)

In [75]:
labels_data['Q19'] =labels_data['Q19'].cat.rename_categories({'Plus de mal que de bien': "Plus de mal que de bien"
                                                              , 'Plutôt pas d’accord': "Plutôt pas d’accord"
                                                              , 'A peu près autant de bien que de mal' : "A peu près autant de bien que de mal"
                                                              , 'Plus de bien que de mal' : "Plus de bien que de mal"
                                                              , 'Vous n&apos;avez pas d’avis' : "Vous n’avez pas d’avis"
                                                              , 'Vous ne souhaitez pas répondre' : "Vous ne souhaitez pas répondre"})

In [76]:
# Variable 'Q20'
# Question: D’après vous, les connaissances actuelles permettent-elles de trancher le débat scientifique autour de l’efficacité de la chloroquine et de ses dérivés (comme l'hydroxychloroquine) contre la COVID-19 ?
labels_data['Q20'].value_counts()

Q20
Non, vous pensez que les connaissances sont encore insuffisantes                                             815
Oui, et vous pensez que la chloroquine et ses dérivés ne sont pas un bon traitement contre le coronavirus    458
Vous n&apos;avez pas d’avis                                                                                  403
Oui, et vous pensez que la chloroquine et ses dérivés sont un bon traitement contre le coronavirus           309
Vous ne souhaitez pas répondre                                                                                37
Name: count, dtype: int64

In [77]:
#changer l'order des categories de la variable 'Q20'
labels_data['Q20'] = pd.Categorical(labels_data['Q20'], 
                            categories=['Non, vous pensez que les connaissances sont encore insuffisantes'
                                        , 'Oui, et vous pensez que la chloroquine et ses dérivés ne sont pas un bon traitement contre le coronavirus'
                                        , 'Oui, et vous pensez que la chloroquine et ses dérivés sont un bon traitement contre le coronavirus'
                                        ,'Vous n&apos;avez pas d’avis'
                                       , 'Vous ne souhaitez pas répondre'],
                            ordered=True)

In [78]:
labels_data['Q20'] =labels_data['Q20'].cat.rename_categories({'Non, vous pensez que les connaissances sont encore insuffisantes': "Non, vous pensez que les connaissances sont encore insuffisantes"
                                                              , 'Plutôt pas d’accord': "Plutôt pas d’accord"
                                                              , 'Oui, et vous pensez que la chloroquine et ses dérivés ne sont pas un bon traitement contre le coronavirus' : "Oui, et vous pensez que la chloroquine et ses dérivés ne sont pas un bon traitement contre le coronavirus"
                                                              , 'Oui, et vous pensez que la chloroquine et ses dérivés sont un bon traitement contre le coronavirus' : "Oui, et vous pensez que la chloroquine et ses dérivés sont un bon traitement contre le coronavirus"
                                                              , 'Vous n&apos;avez pas d’avis' : "Vous n’avez pas d’avis"
                                                              , 'Vous ne souhaitez pas répondre' : "Vous ne souhaitez pas répondre"})

In [79]:
# Variable 'Q21'
# Question: Comment est votre état de santé en général ?
labels_data['Q21'].value_counts()

Q21
Bon             936
Assez bon       577
Très bon        327
Mauvais         164
Très mauvais     18
Name: count, dtype: int64

In [80]:
#changer l'order des categories de la variable 'Q21'
labels_data['Q21'] = pd.Categorical(labels_data['Q21'], 
                            categories=['Très mauvais', 'Mauvais', 'Assez bon'
                                        ,'Bon', 'Très bon'], 
                            ordered=True)

In [81]:
# Variable 'Q22'
# Question: Avez-vous une maladie ou un problème de santé qui soit chronique ou de caractère durable ?
labels_data['Q22'].value_counts()

Q22
Non                                       1027
Oui, j’ai un problème de santé             725
Oui, j’ai plusieurs problèmes de santé     270
Name: count, dtype: int64

In [82]:
#changer l'order des categories de la variable 'Q22'
labels_data['Q22'] = pd.Categorical(labels_data['Q22'], 
                            categories=['Non', 'Oui, j’ai un problème de santé'
                                        ,'Oui, j’ai plusieurs problèmes de santé'], 
                            ordered=True)

In [83]:
# Variable 'Q23_A'
# Question: Pour vous, est-il facile ou difficile de payer toutes les factures à la fin du mois ?
labels_data['Q23_A'].value_counts()

Q23_A
Facile            1005
Difficile          628
Très facile        253
Très difficile     136
Name: count, dtype: int64

In [84]:
# Variable 'Q23_B'
# Question: Pour vous, est-il facile ou difficile d'acheter des médicaments si vous en avez besoin? (les médicaments prescrits qui ne sont pas ou que partiellement remboursés par l'assurance-maladie)
labels_data['Q23_B'].value_counts()

Q23_B
Facile            1085
Difficile          552
Très facile        300
Très difficile      85
Name: count, dtype: int64

In [85]:
# Variable 'Q23_C'
# Question: Payer les examens médicaux et les traitements si vous en avez besoin ? (comme des traitements dentaires, des lunettes ou des traitements de médecine alternative, des mesures thérapeutiques ou des moyens auxiliaires)
labels_data['Q23_C'].value_counts()

Q23_C
Facile            868
Difficile         713
Très facile       247
Très difficile    194
Name: count, dtype: int64

In [86]:
Items23 = ['Q23_A', 'Q23_B', 'Q23_C']

In [87]:
for i in labels_data[Items23].columns:
    labels_data[i] = pd.Categorical(labels_data[i], 
                            categories=['Très facile', 'Facile'
                                        ,'Difficile', 'Très difficile'] 
                                        , ordered=True)

In [88]:
# Variable 'Q24_A'
# Question: Dans quelle mesure êtes-vous d’accord ou pas d’accord avec chacune des déclarations suivantes ? Les médecines alternatives/naturelles offrent de meilleures solutions aux problèmes de santé que la médecine conventionnelle
labels_data['Q24_A'].value_counts()

Q24_A
Ni d’accord, ni pas d’accord      773
Plutôt d’accord                   432
Plutôt pas d’accord               329
Tout à fait d’accord              187
Pas d’accord du tout              174
Vous n&apos;avez pas d’avis       117
Vous ne souhaitez pas répondre     10
Name: count, dtype: int64

In [89]:
# Variable 'Q24_B'
# Question: Dans quelle mesure êtes-vous d’accord ou pas d’accord avec chacune des déclarations suivantes ? Les médecines alternatives/naturelles promettent plus qu’elles ne peuvent offrir
labels_data['Q24_B'].value_counts()

Q24_B
Plutôt d’accord                   677
Ni d’accord, ni pas d’accord      619
Plutôt pas d’accord               252
Tout à fait d’accord              237
Vous n&apos;avez pas d’avis       143
Pas d’accord du tout               84
Vous ne souhaitez pas répondre     10
Name: count, dtype: int64

In [90]:
# Variable 'Q24_C'
# Question: Dans quelle mesure êtes-vous d’accord ou pas d’accord avec chacune des déclarations suivantes ? Il y a moins d’effets secondaires avec les médecines alternatives/naturelles.
labels_data['Q24_C'].value_counts()

Q24_C
Plutôt d’accord                   795
Ni d’accord, ni pas d’accord      502
Tout à fait d’accord              362
Vous n&apos;avez pas d’avis       161
Plutôt pas d’accord               131
Pas d’accord du tout               54
Vous ne souhaitez pas répondre     17
Name: count, dtype: int64

In [91]:
# Variable 'Q24_D'
# Question: Dans quelle mesure êtes-vous d’accord ou pas d’accord avec chacune des déclarations suivantes ? Les médecines alternatives/naturelles, à base de plantes, sont plus saines que les médicaments prescrits par le médecin
labels_data['Q24_D'].value_counts()

Q24_D
Plutôt d’accord                   757
Ni d’accord, ni pas d’accord      569
Tout à fait d’accord              342
Plutôt pas d’accord               149
Vous n&apos;avez pas d’avis       123
Pas d’accord du tout               68
Vous ne souhaitez pas répondre     14
Name: count, dtype: int64

In [92]:
# Variable 'Q24_E'
# Question: Dans quelle mesure êtes-vous d’accord ou pas d’accord avec chacune des déclarations suivantes ? Les médecines alternatives/naturelles renforcent le corps et l’aident à se soigner par lui-même
labels_data['Q24_E'].value_counts()

Q24_E
Plutôt d’accord                   721
Ni d’accord, ni pas d’accord      633
Tout à fait d’accord              259
Plutôt pas d’accord               160
Vous n&apos;avez pas d’avis       156
Pas d’accord du tout               80
Vous ne souhaitez pas répondre     13
Name: count, dtype: int64

In [93]:
# Variable 'Q24_F'
# Question: Dans quelle mesure êtes-vous d’accord ou pas d’accord avec chacune des déclarations suivantes ? Pour être en bonne santé l’esprit est aussi important que le corps.
labels_data['Q24_F'].value_counts()

Q24_F
Tout à fait d’accord              957
Plutôt d’accord                   751
Ni d’accord, ni pas d’accord      169
Vous n&apos;avez pas d’avis        72
Plutôt pas d’accord                39
Pas d’accord du tout               24
Vous ne souhaitez pas répondre     10
Name: count, dtype: int64

In [94]:
# Variable 'Q24_G'
# Question: Dans quelle mesure êtes-vous d’accord ou pas d’accord avec chacune des déclarations suivantes ? Une nourriture saine est plus efficace que les médicaments pour aider le corps à lutter contre les maladies
labels_data['Q24_G'].value_counts()

Q24_G
Plutôt d’accord                   795
Ni d’accord, ni pas d’accord      516
Tout à fait d’accord              420
Plutôt pas d’accord               134
Vous n&apos;avez pas d’avis        88
Pas d’accord du tout               58
Vous ne souhaitez pas répondre     11
Name: count, dtype: int64

In [95]:
# Variable 'Q24_H'
# Question: Dans quelle mesure êtes-vous d’accord ou pas d’accord avec chacune des déclarations suivantes ? Il existe des produits naturels meilleurs que les vaccins pour booster son système immunitaire
labels_data['Q24_H'].value_counts()

Q24_H
Ni d’accord, ni pas d’accord      558
Plutôt d’accord                   458
Plutôt pas d’accord               293
Tout à fait d’accord              275
Pas d’accord du tout              237
Vous n&apos;avez pas d’avis       181
Vous ne souhaitez pas répondre     20
Name: count, dtype: int64

In [96]:
# Variable 'Q24_I'
# Question: Dans quelle mesure êtes-vous d’accord ou pas d’accord avec chacune des déclarations suivantes ? Les médecines alternatives/naturelles ne doivent venir qu’en complément de la médecine conventionnelle
labels_data['Q24_I'].value_counts()

Q24_I
Plutôt d’accord                   829
Ni d’accord, ni pas d’accord      444
Tout à fait d’accord              437
Plutôt pas d’accord               134
Vous n&apos;avez pas d’avis       102
Pas d’accord du tout               66
Vous ne souhaitez pas répondre     10
Name: count, dtype: int64

In [97]:
Items24 = ['Q24_A', 'Q24_B', 'Q24_C', 'Q24_D', 'Q24_E', 'Q24_F', 'Q24_G', 'Q24_H', 'Q24_I']

In [98]:
for i in labels_data[Items24].columns:
    labels_data[i] = pd.Categorical(labels_data[i], 
                            categories=['Pas d’accord du tout', 'Plutôt pas d’accord'
                                        ,'Ni d’accord, ni pas d’accord', 'Plutôt d’accord'
                                        , 'Tout à fait d’accord'
                                        , 'Vous n&apos;avez pas d’avis'
                                        , 'Vous ne souhaitez pas répondre'] 
                                        , ordered=True)

In [99]:
# Variable 'Q29'
# Question: Comptez-vous vous faire vacciner contre le coronavirus à un moment ou un autre ?
labels_data['Q29'].value_counts()

Q29
Vous êtes déjà vacciné (une dose ou plus)    1401
Oui certainement                              294
Non, certainement pas                         130
Oui, probablement                             121
Non, probablement pas                          76
Name: count, dtype: int64

In [100]:
labels_data['Q29'] = pd.Categorical(labels_data['Q29'], 
                            categories=['Vous êtes déjà vacciné (une dose ou plus)', 'Non, certainement pas'
                                        , 'Non, probablement pas', 'Oui, probablement'
                                        , 'Oui certainement'], 
                            ordered=True)

In [101]:
# Variable 'Q30'
# Question: Êtes-vous très, plutôt, plutôt pas ou pas du tout favorable aux vaccins en général ?
labels_data['Q30'].value_counts()

Q30
Plutôt favorable                  921
Très favorable                    675
Plutôt pas favorable              219
Pas du tout favorable             108
Vous n’avez pas d’avis             69
Vous ne souhaitez pas répondre     30
Name: count, dtype: int64

In [102]:
#changer l'order des categories de la variable 'Q30'
labels_data['Q30'] = pd.Categorical(labels_data['Q30'], 
                            categories=['Pas du tout favorable', 'Plutôt pas favorable' ,'Plutôt favorable'
                                        ,'Très favorable', 'Vous n’avez pas d’avis'
                                       , 'Vous ne souhaitez pas répondre'], 
                            ordered=True)

In [103]:
# Variable 'Q31'
# Question: Êtes-vous favorable au vaccin contre la grippe ?
labels_data['Q31'].value_counts()

Q31
Oui, tout à fait                  709
Oui, plutôt                       631
Non, plutôt pas                   343
Non, pas du tout                  187
Vous n’avez pas d’avis            136
Vous ne souhaitez pas répondre     16
Name: count, dtype: int64

In [104]:
# Variable 'Q32'
# Question: Êtes-vous favorable au vaccin contre les papillomavirus humains (HPV) ?
labels_data['Q32'].value_counts()

Q32
Oui, tout à fait                  669
Oui, plutôt                       654
Vous n’avez pas d’avis            424
Non, plutôt pas                   152
Non, pas du tout                   91
Vous ne souhaitez pas répondre     32
Name: count, dtype: int64

In [105]:
# Variable 'Q33'
# Question: Êtes-vous favorable au vaccin contre la rougeole ?
labels_data['Q33'].value_counts()

Q33
Oui, tout à fait                  1014
Oui, plutôt                        720
Vous n’avez pas d’avis             146
Non, plutôt pas                     89
Non, pas du tout                    34
Vous ne souhaitez pas répondre      19
Name: count, dtype: int64

In [106]:
# Variable 'Q34'
# Question: Êtes-vous favorable au vaccin contre l’hépatite B ?
labels_data['Q34'].value_counts()

Q34
Oui, tout à fait                  758
Oui, plutôt                       713
Vous n’avez pas d’avis            219
Non, plutôt pas                   194
Non, pas du tout                  106
Vous ne souhaitez pas répondre     32
Name: count, dtype: int64

In [107]:
vaccin= ['Q31', 'Q32', 'Q33', 'Q34']

In [108]:
for i in labels_data[vaccin].columns:
    labels_data[i] = pd.Categorical(labels_data[i], 
                            categories=['Non, pas du tout', 'Non, plutôt pas'
                                        ,'Oui, plutôt', 'Oui, tout à fait'
                                        , 'Vous n’avez pas d’avis'
                                        , 'Vous ne souhaitez pas répondre'] 
                                        , ordered=True)

In [109]:
#Typologie hésitation vaccinale pour touts les vaccins:
# Il s’agit d’une variable score produite à partir de variables suivantes :
# variable 'Q30' : Question : Êtes-vous très, plutôt, plutôt pas ou pas du tout favorable aux vaccinations en général ?
# variable 'Q31' : Question : Êtes-vous favorable au vaccin contre l’hépatite B ?
# variable 'Q32' : Question : Êtes-vous favorable au vaccin contre la grippe ?
# variable 'Q33' : Question : Êtes-vous favorable au vaccin contre les papillomavirus humains (HPV) ?
# variable 'Q34' : Question : Êtes-vous favorable au vaccin contre la rougeole ?


In [110]:
# définir une fonction pour recoder la variable 'Q30':
def vaccinaltyQ30(x):
    if x == 'Plutôt favorable':
        return 'Favorable'
    if x == 'Très favorable':
        return 'Favorable'
    if x == 'Plutôt pas favorable':
        return 'Défavorable'
    if x == 'Pas du tout favorable':
        return 'Défavorable'
    if x == 'Vous n’avez pas d’avis' or x == 'Vous ne souhaitez pas répondre' :
        return 'Je ne sais pas'

In [111]:
labels_data['Q30_cat']= labels_data['Q30'].apply(lambda x : vaccinaltyQ30(x))

In [112]:
labels_data['Q30_cat'].value_counts()

Q30_cat
Favorable         1596
Défavorable        327
Je ne sais pas      99
Name: count, dtype: int64

In [113]:
# définir une fonction pour recoder la variable 'Q31':
def vaccinaltyQ31(x):
    if x == 'Oui, plutôt':
        return 'Favorable'
    if x == 'Oui, tout à fait':
        return 'Favorable'
    if x == 'Non, plutôt pas':
        return 'Défavorable'
    if x == 'Non, pas du tout':
        return 'Défavorable'
    if x == 'Vous n’avez pas d’avis' or x == 'Vous ne souhaitez pas répondre' :
        return 'Je ne sais pas'

In [114]:
labels_data['Q31_cat']= labels_data['Q31'].apply(lambda x : vaccinaltyQ31(x))
labels_data['Q32_cat']= labels_data['Q32'].apply(lambda x : vaccinaltyQ31(x))
labels_data['Q33_cat']= labels_data['Q33'].apply(lambda x : vaccinaltyQ31(x))
labels_data['Q34_cat']= labels_data['Q34'].apply(lambda x : vaccinaltyQ31(x))

In [115]:
# creer une colonne typologie d'hésitation vaccinale en 3 groupes:
# List of conditions
conditions = [
     (labels_data['Q30_cat'] == 'Favorable') &  (labels_data['Q31_cat'] == 'Favorable') & 
     (labels_data['Q32_cat'] == 'Favorable') & (labels_data['Q33_cat'] == 'Favorable') &
     (labels_data['Q34_cat'] == 'Favorable')
    ,(labels_data['Q30_cat'] == 'Favorable') & ((labels_data['Q31_cat'] != 'Favorable')| 
     (labels_data['Q32_cat'] != 'Favorable') | (labels_data['Q33_cat'] != 'Favorable')|
     (labels_data['Q34_cat'] != 'Favorable'))
    ,(labels_data['Q30_cat'] == 'Défavorable') | (labels_data['Q30_cat'] == 'Je ne sais pas')
    
   
]# List of values to return
choices  = [
      "Non-hesitan"
    , "Pro-vaccination mais sélectivement hésitant"
    , "Défavorable à la vaccination en général"
]# create a new column in the DF based on the conditions
labels_data["Ty_hes_vaccinale"] = np.select(conditions, choices)

In [116]:
labels_data["Ty_hes_vaccinale"].value_counts(normalize = True)*100

Ty_hes_vaccinale
Non-hesitan                                      41.889219
Pro-vaccination mais sélectivement hésitant    37.042532
Défavorable à la vaccination en général          21.068249
Name: proportion, dtype: float64

In [117]:
#changer l'order des categories de la variable 'Ty_hes_vaccinale'
labels_data['Ty_hes_vaccinale'] = pd.Categorical(labels_data['Ty_hes_vaccinale'], 
                            categories=['Défavorable à la vaccination en général'
                                        ,'Pro-vaccination mais sélectivement hésitant'
                                        ,'Non-hesitan'], 
                            ordered=True)

In [118]:
# Variable 'Q36_A'
# Question: Dans quelle mesure il est facile pour vous de comprendre le fonctionnement du système de santé ?
labels_data['Q36_A'].value_counts()

Q36_A
Facile            1106
Difficile          547
Très facile        289
Très difficile      80
Name: count, dtype: int64

In [119]:
# Variable 'Q36_B'
# Question: Dans quelle mesure il est facile pour vous de savoir à quel type de service médical ou prestataire de soins particulier vous adresser en cas de problème de santé ?
labels_data['Q36_B'].value_counts()

Q36_B
Facile            1200
Très facile        386
Difficile          379
Très difficile      57
Name: count, dtype: int64

In [120]:
# Variable 'Q36_C'
# Question: Dans quelle mesure il est facile pour vous de savoir dans quelle mesure votre assurance-maladie prend en charge les coûts de certaines prestations ?
labels_data['Q36_C'].value_counts()

Q36_C
Facile            866
Difficile         750
Très facile       229
Très difficile    177
Name: count, dtype: int64

In [121]:
# Variable 'Q36_D'
# Question: Dans quelle mesure il est facile pour vous de comprendre les réformes en cours du système de santé qui pourraient affecter vos soins ?
labels_data['Q36_D'].value_counts()

Q36_D
Difficile         1000
Facile             599
Très difficile     297
Très facile        126
Name: count, dtype: int64

In [122]:
# Variable 'Q36_E'
# Question: Dans quelle mesure il est facile pour vous de vous renseigner sur vos droits en tant que patient ou utilisateur du système de santé ?
labels_data['Q36_E'].value_counts()

Q36_E
Facile            922
Difficile         732
Très facile       229
Très difficile    139
Name: count, dtype: int64

In [123]:
# Variable 'Q36_F'
# Question: Dans quelle mesure il est facile pour vous de choisir un service médical ou un prestataire de soins en particulier ?
labels_data['Q36_F'].value_counts()

Q36_F
Facile            910
Difficile         740
Très facile       226
Très difficile    146
Name: count, dtype: int64

In [124]:
# Variable 'Q36_G'
# Question: Dans quelle mesure il est facile pour vous de trouver des informations sur la qualité d'un service médical ou d'un prestataire de soins particulier ?
labels_data['Q36_G'].value_counts()

Q36_G
Difficile         872
Facile            788
Très facile       196
Très difficile    166
Name: count, dtype: int64

In [125]:
# Variable 'Q36_H'
# Question: Dans quelle mesure il est facile pour vous de déterminer si un service médical ou un prestataire de soins particulier répondra à vos attentes et vos souhaits en matière de soins ?
labels_data['Q36_H'].value_counts()

Q36_H
Difficile         962
Facile            759
Très difficile    157
Très facile       144
Name: count, dtype: int64

In [126]:
# Variable 'Q36_I'
# Question: Dans quelle mesure il est facile pour vous de comprendre comment prendre un rendez-vous auprès d'un service médical ou d'un prestataire de soins ?
labels_data['Q36_I'].value_counts()

Q36_I
Facile            1178
Très facile        416
Difficile          358
Très difficile      70
Name: count, dtype: int64

In [127]:
# Variable 'Q36_J'
# Question: Dans quelle mesure il est facile pour vous de vous renseigner sur les possibilités de soutien qui existent pour vous aider à vous orienter dans le système de santé ?
labels_data['Q36_J'].value_counts()

Q36_J
Difficile         932
Facile            762
Très difficile    165
Très facile       163
Name: count, dtype: int64

In [128]:
# Variable 'Q36_K'
# Question: Dans quelle mesure il est facile pour vous de trouver le bon interlocuteur pour vos besoins au sein d'un établissement de santé ?
labels_data['Q36_K'].value_counts()

Q36_K
Facile            839
Difficile         833
Très facile       188
Très difficile    162
Name: count, dtype: int64

In [129]:
# Variable 'Q36_L'
# Question: Dans quelle mesure il est facile pour vous de vous défendre si les soins que vous recevez ne répondent pas à vos besoins ?
labels_data['Q36_L'].value_counts()

Q36_L
Difficile         983
Facile            490
Très difficile    419
Très facile       130
Name: count, dtype: int64

In [130]:
Items36 = ['Q36_A', 'Q36_B', 'Q36_C', 'Q36_D', 'Q36_E', 'Q36_F', 'Q36_G', 'Q36_H', 'Q36_I', 'Q36_J', 'Q36_K', 'Q36_L']

In [131]:
for i in labels_data[Items36].columns:
    labels_data[i] = pd.Categorical(labels_data[i], 
                            categories=['Très facile', 'Facile'
                                        ,'Difficile', 'Très difficile'] 
                                        , ordered=True)

In [132]:
# Variable 'Q37_A'
# Question: Dans quelle mesure il est facile pour vous de savoir où obtenir de l'aide d'un professionnel de santé quand vous êtes malade ?
labels_data['Q37_A'].value_counts()

Q37_A
Facile            1146
Difficile          443
Très facile        357
Très difficile      76
Name: count, dtype: int64

In [133]:
# Variable 'Q37_B'
# Question: Dans quelle mesure il est facile pour vous de comprendre les informations sur ce qu'il faut faire en cas d'urgence médicale ?
labels_data['Q37_B'].value_counts()

Q37_B
Facile            1090
Difficile          605
Très facile        252
Très difficile      75
Name: count, dtype: int64

In [134]:
# Variable 'Q37_C'
# Question: Dans quelle mesure il est facile pour vous d'évaluer les risques et bénéfices des différents traitements médicaux possibles ?
labels_data['Q37_C'].value_counts()

Q37_C
Difficile         945
Facile            723
Très difficile    195
Très facile       159
Name: count, dtype: int64

In [135]:
# Variable 'Q37_D'
# Question: Dans quelle mesure il est facile pour vous de suivre les consignes de votre médecin ou pharmacien ?
labels_data['Q37_D'].value_counts()

Q37_D
Facile            1184
Très facile        653
Difficile          154
Très difficile      31
Name: count, dtype: int64

In [136]:
# Variable 'Q37_E'
# Question: Dans quelle mesure il est facile pour vous de trouver des informations sur comment faire en cas de problèmes psychologiques ?
labels_data['Q37_E'].value_counts()

Q37_E
Facile            905
Difficile         772
Très facile       205
Très difficile    140
Name: count, dtype: int64

In [137]:
# Variable 'Q37_F'
# Question: Dans quelle mesure il est facile pour vous de comprendre les informations sur les dépistages et examens recommandés ?
labels_data['Q37_F'].value_counts()

Q37_F
Facile            1114
Difficile          499
Très facile        326
Très difficile      83
Name: count, dtype: int64

In [138]:
# Variable 'Q37_G'
# Question: Dans quelle mesure il est facile pour vous d'évaluer la fiabilité des informations concernant l'impact sur la santé de certains comportements comme fumer, ne pas faire assez d'exercices ou trop boire d'alcool ?
labels_data['Q37_G'].value_counts()

Q37_G
Facile            1158
Difficile          401
Très facile        385
Très difficile      78
Name: count, dtype: int64

In [139]:
# Variable 'Q37_H'
# Question: Dans quelle mesure il est facile pour vous de savoir comment vous protéger des maladies à partir des informations disponibles dans les médias ?
labels_data['Q37_H'].value_counts()

Q37_H
Facile            1053
Difficile          631
Très facile        228
Très difficile     110
Name: count, dtype: int64

In [140]:
# Variable 'Q37_I'
# Question: Dans quelle mesure il est facile pour vous de trouver des informations sur les modes de vie sains, tels que l'activité physique ou une alimentation équilibrée ?
labels_data['Q37_I'].value_counts()

Q37_I
Facile            1241
Très facile        401
Difficile          318
Très difficile      62
Name: count, dtype: int64

In [141]:
# Variable 'Q37_J'
# Question: Dans quelle mesure il est facile pour vous de comprendre les conseils de votre famille ou de vos amis en matière de santé ?
labels_data['Q37_J'].value_counts()

Q37_J
Facile            1277
Très facile        350
Difficile          339
Très difficile      56
Name: count, dtype: int64

In [142]:
# Variable 'Q37_K'
# Question: Dans quelle mesure il est facile pour vous d'évaluer comment vos conditions de logement peuvent affecter votre santé et votre bien-être ?
labels_data['Q37_K'].value_counts()

Q37_K
Facile            1134
Difficile          486
Très facile        328
Très difficile      74
Name: count, dtype: int64

In [143]:
# Variable 'Q37_L'
# Question: Dans quelle mesure il est facile pour vous de prendre des décisions pour améliorer votre santé et votre bien-être ?
labels_data['Q37_L'].value_counts()

Q37_L
Facile            1174
Difficile          458
Très facile        328
Très difficile      62
Name: count, dtype: int64

In [144]:
Items37 = ['Q37_A', 'Q37_B', 'Q37_C', 'Q37_D', 'Q37_E', 'Q37_F', 'Q37_G', 'Q36_H', 'Q37_I', 'Q37_J', 'Q37_K', 'Q37_L']

In [145]:
for i in labels_data[Items37].columns:
    labels_data[i] = pd.Categorical(labels_data[i], 
                            categories=['Très facile', 'Facile'
                                        ,'Difficile', 'Très difficile'] 
                                        , ordered=True)

In [146]:
# Variable 'Q38M1'
# Question: Habituellement, pour vous informer, quel(s) moyen(s) utilisez-vous de préférence ? Item le plus important
labels_data['Q38M1'].value_counts()

Q38M1
La télévision                                              638
Les échanges avec vos proches                              412
D’autres sites Internet                                    270
La radio                                                   213
Les sites de médias sur Internet ou leur compte Twitter    207
La presse écrite imprimée                                  193
Les réseaux sociaux (Facebook, Twitter, etc.)               89
Name: count, dtype: int64

In [147]:
# Variable 'Q38M2'
# Question: Habituellement, pour vous informer, quel(s) moyen(s) utilisez-vous de préférence ? Second item le plus important
labels_data['Q38M2'].value_counts()

Q38M2
La télévision                                              394
Les échanges avec vos proches                              343
D’autres sites Internet                                    317
La radio                                                   288
La presse écrite imprimée                                  232
Les sites de médias sur Internet ou leur compte Twitter    167
Les réseaux sociaux (Facebook, Twitter, etc.)              147
#NULL!                                                     134
Name: count, dtype: int64

In [148]:
# Variable 'Q38M3'
# Question: Habituellement, pour vous informer, quel(s) moyen(s) utilisez-vous de préférence ? Troisième item le plus important
labels_data['Q38M3'].value_counts()

Q38M3
#NULL!                                                     402
Les échanges avec vos proches                              367
La télévision                                              282
D’autres sites Internet                                    263
La presse écrite imprimée                                  216
La radio                                                   173
Les sites de médias sur Internet ou leur compte Twitter    173
Les réseaux sociaux (Facebook, Twitter, etc.)              146
Name: count, dtype: int64

In [149]:
# creer une colonne de Reseaux sociaux placement en 4 groupes:
# List of conditions
conditions = [
     (labels_data['Q38M1'] == 'Les réseaux sociaux (Facebook, Twitter, etc.)') & (labels_data['Q38M2'] != 'Les réseaux sociaux (Facebook, Twitter, etc.)') & 
     (labels_data['Q38M3'] != 'Les réseaux sociaux (Facebook, Twitter, etc.)')
    ,(labels_data['Q38M1'] != 'Les réseaux sociaux (Facebook, Twitter, etc.)') & (labels_data['Q38M2'] == 'Les réseaux sociaux (Facebook, Twitter, etc.)') &
     (labels_data['Q38M3'] != 'Les réseaux sociaux (Facebook, Twitter, etc.)') 
    ,(labels_data['Q38M1'] != 'Les réseaux sociaux (Facebook, Twitter, etc.)') & (labels_data['Q38M2'] != 'Les réseaux sociaux (Facebook, Twitter, etc.)') &
     (labels_data['Q38M3'] == 'Les réseaux sociaux (Facebook, Twitter, etc.)')
    ,(labels_data['Q38M1'] != 'Les réseaux sociaux (Facebook, Twitter, etc.)') & (labels_data['Q38M2'] != 'Les réseaux sociaux (Facebook, Twitter, etc.)') &
     (labels_data['Q38M3'] != 'Les réseaux sociaux (Facebook, Twitter, etc.)') 
   
]# List of values to return
choices  = [
      "Première source" 
    , "Deuxième source"
    , "Troisième source"
    , "Pas dans les 3 premières"
]# create a new column in the data based on the conditions
labels_data["Reseaux_sociaux_placement"] = np.select(conditions, choices)

In [150]:
labels_data["Reseaux_sociaux_placement"].value_counts()

Reseaux_sociaux_placement
Pas dans les 3 premières    1640
Deuxième source              147
Troisième source             146
Première source               89
Name: count, dtype: int64

In [151]:
#changer l'order des categories de la variable 'Reseaux_sociaux_placement'
labels_data['Reseaux_sociaux_placement'] = pd.Categorical(labels_data['Reseaux_sociaux_placement'], 
                            categories=['Pas dans les 3 premières', 'Première source', 'Deuxième source'
                                        , 'Troisième source'], 
                            ordered=True)

In [152]:
# Variable 'Q39'
# Question: Diriez-vous que vous vous intéressez à la politique ?
labels_data['Q39'].value_counts()

Q39
Oui, un peu                       782
Non, pas vraiment                 472
Oui, beaucoup                     393
Non, pas du tout                  357
Vous ne souhaitez pas répondre     18
Name: count, dtype: int64

In [153]:
#changer l'order des categories de la variable'Q39'
labels_data['Q39'] = pd.Categorical(labels_data['Q39'], 
                            categories=['Non, pas du tout', 'Non, pas vraiment'
                                        ,'Oui, un peu', 'Oui, beaucoup'
                                       , 'Vous ne souhaitez pas répondre'], 
                            ordered=True)

In [154]:
# Variable 'Q40'
# Question: A quelle fréquence suivez-vous l’actualité politique ?
labels_data['Q40'].value_counts()

Q40
Tous les jours ou presque      647
Moins souvent                  454
Plusieurs fois par semaine     412
Une à deux fois par semaine    307
Jamais                         202
Name: count, dtype: int64

In [155]:
#changer l'order des categories de la variable'Q40'
labels_data['Q40'] = pd.Categorical(labels_data['Q40'], 
                            categories=['Jamais', 'Moins souvent'
                                        ,'Une à deux fois par semaine'
                                        , 'Plusieurs fois par semaine'
                                        , 'Tous les jours ou presque'], 
                            ordered=True)

In [156]:
# Variable 'Q41'
# Question: A quelle fréquence participez-vous aux élections ?
labels_data['Q41'].value_counts()

Q41
Vous votez à toutes les élections        1084
Vous votez à la plupart des élections     422
Vous votez à certaines élections          227
Vous ne votez pas                         138
Vous votez rarement                       104
Vous ne souhaitez pas répondre             47
Name: count, dtype: int64

In [157]:
#changer l'order des categories de la variable'Q41'
labels_data['Q41'] = pd.Categorical(labels_data['Q41'], 
                            categories=['Vous ne votez pas', 'Vous votez rarement'
                                        ,'Vous votez à certaines élections'
                                        , 'Vous votez à la plupart des élections'
                                        , 'Vous votez à toutes les élections'
                                       , 'Vous ne souhaitez pas répondre'], 
                            ordered=True)

In [158]:
# Compétence politique : Il s’agit d’une variable score produite à partir de 3 questions :
# Q39 : « Diriez-vous que vous vous intéressez à la politique ? Oui, beaucoup/Oui, un peu/Non, pas vraiment/Non, pas du tout »
# Q40 : « A quelle fréquence suivez-vous l’actualité politique ? Tous les jours ou presque/Plusieurs fois par semaine/Une à deux fois par semaine/Moins souvent/Jamais »
# Q41 : « A quelle fréquence participez-vous aux élections ?
# Vous votez à toutes les élections/Vous votez à la plupart des élections/Vous votez à certaines élections/Vous votez rarement/Vous ne votez pas »

# Les questions ont été recodées en numérique avec des scores allant de 0 (niveau minimal de compétence politique) à 4 (plus haut niveau).
# Nous avons ensuite réalisé une ACM et récupéré les coordonnés des individus sur le premier axe factoriel. 


In [159]:
# définir une fonction pour recoder la variable 'Q39':
def competpolitique_Q39(x):
    if x == 'Non, pas du tout':
        return 0
    if x == 'Non, pas vraiment':
        return 1
    if x == 'Oui, un peu':
        return 2
    if x == 'Oui, beaucoup':
        return 3
    if x == 'Vous ne souhaitez pas répondre':
        return 1.5

In [160]:
labels_data['Q39_cat']= labels_data['Q39'].apply(lambda x : competpolitique_Q39(x))

In [161]:
labels_data['Q39_cat'].value_counts()

Q39_cat
2.0    782
1.0    472
3.0    393
0.0    357
1.5     18
Name: count, dtype: int64

In [162]:
# définir une fonction pour recoder la variable 'Q40':
def competpolitique_Q40(x):
    if x == 'Jamais':
        return 0
    if x == 'Moins souvent':
        return 1
    if x == 'Une à deux fois par semaine':
        return 2
    if x == 'Plusieurs fois par semaine':
        return 3
    if x == 'Tous les jours ou presque':
        return 4

In [163]:
labels_data['Q40_cat']= labels_data['Q40'].apply(lambda x : competpolitique_Q40(x))

In [164]:
labels_data['Q40_cat'].value_counts()

Q40_cat
4    647
1    454
3    412
2    307
0    202
Name: count, dtype: int64

In [165]:
# définir une fonction pour recoder la variable 'Q41':
def competpolitique_Q41(x):
    if x == 'Vous ne votez pas':
        return 0
    if x == 'Vous votez rarement':
        return 1
    if x == 'Vous votez à certaines élections':
        return 2
    if x == 'Vous votez à la plupart des élections':
        return 3
    if x == 'Vous votez à toutes les élections':
        return 4
    if x == 'Vous ne souhaitez pas répondre':
        return 1.5

In [166]:
labels_data['Q41_cat']= labels_data['Q41'].apply(lambda x : competpolitique_Q41(x))

In [167]:
labels_data['Q41_cat'].value_counts()

Q41_cat
4.0    1084
3.0     422
2.0     227
0.0     138
1.0     104
1.5      47
Name: count, dtype: int64

In [168]:
labels_data[['Q39_cat', 'Q40_cat', 'Q41_cat']] = labels_data[['Q39_cat', 'Q40_cat', 'Q41_cat']].astype('category')

In [169]:
competpol= ['Q39_cat', 'Q40_cat', 'Q41_cat']

In [170]:
# Application de l'ACM
acm_competpol = prince.MCA(n_components = 2)

acm_competpol = acm_competpol.fit(labels_data[competpol])

ax = acm_competpol.plot(X=labels_data[competpol])
ax


alt.Chart(...)

In [171]:
competpol_s= acm_competpol.row_coordinates(labels_data)
competpol_s.rename(columns = {0:'Compoenent 0', 1 : 'Componenet 1'}, inplace = True)

competpol_score = pd.concat([labels_data, competpol_s], axis = 1)

In [172]:
competpol_score['Compoenent 0'].max()

1.9312601655406143

In [173]:
competpol_score['Compoenent 0'].min()

-1.0205186575236056

In [174]:
competpol_score['Compoenent 0'].quantile([0.20, 0.40, 0.60, 0.80])

0.2   -0.782384
0.4   -0.372856
0.6    0.065003
0.8    0.848082
Name: Compoenent 0, dtype: float64

In [175]:
competpol_score.loc[competpol_score['Compoenent 0'] < -0.782384 ]
# Compétance très élevée

,CaseId,STATUT,poids_init,poids_redresses_stand,INT01,Q01,Q01_R0,Q02,Q03,Q04,Q05_A,Q05_B,Q05_R0,Q06,Q07,Q08,Q09,Q10_A,Q10_B,Q10_C,Q10_D,Q10_E,Q11,Q11_1,Q12,Q13,Q14,Q15,Q16_A,Q16_B,Q16_C,Q16_D,Q16_E,Q16_F,Q17_A,Q17_B,Q17_C,Q17_D,Q17_E,Q17_F,Q17_G,Q17_H,Q18,Q19,Q20,Q21,Q22,Q23_A,Q23_B,Q23_C,Q24_A,Q24_B,Q24_C,Q24_D,Q24_E,Q24_F,Q24_G,Q24_H,Q24_I,Q25,Q26,Q27,Q28_A,Q28_B,Q28_C,Q29,Q44,Q30,Q31,Q32,Q33,Q34,Q29_1,Q29_2,Q29_2_PRE,Q29_3,Q35,Q36_A,Q36_B,Q36_C,Q36_D,Q36_E,Q36_F,Q36_G,Q36_H,Q36_I,Q36_J,Q36_K,Q36_L,Q37_A,Q37_B,Q37_C,Q37_D,Q37_E,Q37_F,Q37_G,Q37_H,Q37_I,Q37_J,Q37_K,Q37_L,Q38M1,Q38M2,Q38M3,Q39,Q40,Q41,Q42_A,Q42_B,Q43_A,Q43_B,Q43_C,Q43_D,Q43_E,Q43_F,Q43_G,Q43_H,Q43_I,Q43_J,COMMENTAIRES,Diplome,Q17_A_cat,Q17_B_cat,Q17_C_cat,Q17_D_cat,Q17_E_cat,Q17_F_cat,Q17_G_cat,Q17_H_cat,Ind_CI,Q17_A_cat_No,Q17_B_cat_No,Q17_C_cat_No,Q17_D_cat_No,Q17_E_cat_No,Q17_F_cat_No,Q17_G_cat_No,Q17_H_cat_No,Ind_NCI,Q17_A_cat_NSP,Q17_B_cat_NSP,Q17_C_cat_NSP,Q17_D_cat_NSP,Q17_E_cat_NSP,Q17_F_cat_NSP,Q17_G_cat_NSP,Q17_H_cat_NSP,Ind_CI_NSP,Ind_CI_Cat,Ind_NCI_Cat,Ind_CI_NSP_Cat,Q17_A_cat_clustering,Q17_B_cat_clustering,Q17_C_cat_clustering,Q17_D_cat_clustering,Q17_E_cat_clustering,Q17_F_cat_clustering,Q17_G_cat_clustering,Q17_H_cat_clustering,cat_CI2,cluster_labels,Q30_cat,Q31_cat,Q32_cat,Q33_cat,Q34_cat,Ty_hes_vaccinale,Reseaux_sociaux_placement,Q39_cat,Q40_cat,Q41_cat,Compoenent 0,Componenet 1
0,10,Complet,25416.857072,1.015050,Oui,1946,65-74,Un homme,Provence-Alpes-Côte d&apos;Azur,de 2 000 à moins de 20 000 habitants (exemple ...,Vous êtes à la retraite ou en pré-retraite,Ingénieurs et cadres techniques d&apos;entreprise,Retraités,"Diplôme d’une grande école (ingénieur, commerc...",4000 euros et plus,2,Non,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,France,Oui,NaN,#NULL!,Vous connaissez un peu les essais cliniques,"Oui, plus ou moins",Tout à fait d’accord,Plutôt d’accord,Plutôt d’accord,Plutôt pas d’accord,Tout à fait d’accord,Plutôt pas d’accord,Plutôt,Plutôt,Tout à fait,Pas du tout,Plutôt,Pas du tout,Plutôt,Plutôt pas,Assez,A peu près autant de bien que de mal,"Oui, et vous pensez que la chloroquine et ses ...",Assez bon,"Oui, j’ai un problème de santé",Très facile,Facile,Facile,Tout à fait d’accord,"Ni d’accord, ni pas d’accord",Plutôt d’accord,Plutôt d’accord,Plutôt d’accord,Tout à fait d’accord,Plutôt d’accord,Plutôt d’accord,Plutôt d’accord,Vous connaissez seulement le terme “CBD”,Un peu,Jamais,Jamais,Tous les jours ou presque,Jamais,Vous êtes déjà vacciné (une dose ou plus),Plutôt pas favorable,Plutôt favorable,"Oui, tout à fait","Oui, plutôt","Oui, tout à fait","Oui, tout à fait",Mars 2021,Non,NaN,#NULL!,"Oui, mais vous l&apos;avez désinstallée",Difficile,Facile,Difficile,Très difficile,Difficile,Difficile,Difficile,Difficile,Facile,Difficile,Difficile,Difficile,Facile,Facile,Difficile,Facile,Difficile,Facile,Difficile,Difficile,Facile,Facile,Difficile,Difficile,Les échanges avec vos proches,La presse écrite imprimée,"Les réseaux sociaux (Facebook, Twitter, etc.)","Oui, beaucoup",Tous les jours ou presque,Vous votez à toutes les élections,Les Républicains,#NULL!,Plutôt pas confiance,Pas du tout confiance,Plutôt confiance,Plutôt pas confiance,Plutôt confiance,Plutôt confiance,Pas du tout confiance,Pas du tout confiance,Plutôt confiance,Plutôt confiance,NaN,Master ou plus,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,5.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3-4,1-2,0,D’accord,D’accord,D’accord,Pas d’accord,D’accord,Pas d’accord,D’accord,Pas d’accord,0,Money matters,Favorable,Favorable,Favorable,Favorable,Favorable,Non-hesitan,Troisième source,3.0,4,4.0,-1.020519,0.980862
4,1008,Complet,25416.857072,1.053411,Oui,1971,35-49,Un homme,Pays de la Loire,de 2 000 à moins de 20 000 habitants (exemple ...,Vous exercez une activité professionnelle (act...,Employés civils et agents de service de la fon...,Employés,Baccalauréat technologique ou professionnel,4000 euros et plus,3,Oui,0,0,0,1,0,France,Oui,NaN,#NULL!,Vous connaissez seulement le terme «

In [176]:
competpol_score.loc[competpol_score['Compoenent 0']> 0.848082]
# très faible

,CaseId,STATUT,poids_init,poids_redresses_stand,INT01,Q01,Q01_R0,Q02,Q03,Q04,Q05_A,Q05_B,Q05_R0,Q06,Q07,Q08,Q09,Q10_A,Q10_B,Q10_C,Q10_D,Q10_E,Q11,Q11_1,Q12,Q13,Q14,Q15,Q16_A,Q16_B,Q16_C,Q16_D,Q16_E,Q16_F,Q17_A,Q17_B,Q17_C,Q17_D,Q17_E,Q17_F,Q17_G,Q17_H,Q18,Q19,Q20,Q21,Q22,Q23_A,Q23_B,Q23_C,Q24_A,Q24_B,Q24_C,Q24_D,Q24_E,Q24_F,Q24_G,Q24_H,Q24_I,Q25,Q26,Q27,Q28_A,Q28_B,Q28_C,Q29,Q44,Q30,Q31,Q32,Q33,Q34,Q29_1,Q29_2,Q29_2_PRE,Q29_3,Q35,Q36_A,Q36_B,Q36_C,Q36_D,Q36_E,Q36_F,Q36_G,Q36_H,Q36_I,Q36_J,Q36_K,Q36_L,Q37_A,Q37_B,Q37_C,Q37_D,Q37_E,Q37_F,Q37_G,Q37_H,Q37_I,Q37_J,Q37_K,Q37_L,Q38M1,Q38M2,Q38M3,Q39,Q40,Q41,Q42_A,Q42_B,Q43_A,Q43_B,Q43_C,Q43_D,Q43_E,Q43_F,Q43_G,Q43_H,Q43_I,Q43_J,COMMENTAIRES,Diplome,Q17_A_cat,Q17_B_cat,Q17_C_cat,Q17_D_cat,Q17_E_cat,Q17_F_cat,Q17_G_cat,Q17_H_cat,Ind_CI,Q17_A_cat_No,Q17_B_cat_No,Q17_C_cat_No,Q17_D_cat_No,Q17_E_cat_No,Q17_F_cat_No,Q17_G_cat_No,Q17_H_cat_No,Ind_NCI,Q17_A_cat_NSP,Q17_B_cat_NSP,Q17_C_cat_NSP,Q17_D_cat_NSP,Q17_E_cat_NSP,Q17_F_cat_NSP,Q17_G_cat_NSP,Q17_H_cat_NSP,Ind_CI_NSP,Ind_CI_Cat,Ind_NCI_Cat,Ind_CI_NSP_Cat,Q17_A_cat_clustering,Q17_B_cat_clustering,Q17_C_cat_clustering,Q17_D_cat_clustering,Q17_E_cat_clustering,Q17_F_cat_clustering,Q17_G_cat_clustering,Q17_H_cat_clustering,cat_CI2,cluster_labels,Q30_cat,Q31_cat,Q32_cat,Q33_cat,Q34_cat,Ty_hes_vaccinale,Reseaux_sociaux_placement,Q39_cat,Q40_cat,Q41_cat,Compoenent 0,Componenet 1
1,1000,Complet,25416.857072,0.678083,Oui,2000,18-24,Une femme,Auvergne-Rhône-Alpes,de plus de 100 000 habitants (exemple : Grenob...,Vous exercez une activité professionnelle (act...,Professions intermédiaires administratives et ...,Professions Intermédiaires,"BTS, DUT, DEUST ou diplôme équivalent",1000 à 1500 euros,2,Non,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,France,Oui,NaN,#NULL!,Vous connaissez seulement le terme « essai cli...,"Non, pas du tout",Vous n’avez pas d’avis,Vous n’avez pas d’avis,Vous n’avez pas d’avis,Vous n’avez pas d’avis,Vous n’avez pas d’avis,Vous n’avez pas d’avis,Vous n’avez pas d’avis,Vous n’avez pas d’avis,Vous n’avez pas d’avis,Vous n’avez pas d’avis,Vous n’avez pas d’avis,Vous n’avez pas d’avis,Vous n’avez pas d’avis,Vous n’avez pas d’avis,Peu,A peu près autant de bien que de mal,"Non, vous pensez que les connaissances sont en...",Très bon,Non,Très facile,Très facile,Facile,Plutôt d’accord,"Ni d’accord, ni pas d’accord",Plutôt d’accord,Tout à fait d’accord,Tout à fait d’accord,Tout à fait d’accord,Plutôt d’accord,Tout à fait d’accord,Pas d’accord du tout,Vous n&apos;avez jamais entendu parler du CBD,#NULL!,#NULL!,Jamais,Moins d&apos;une fois par semaine,Jamais,Vous êtes déjà vacciné (une dose ou plus),Pas du tout favorable,Vous ne souhaitez pas répondre,"Oui, plutôt",Vous ne souhaitez pas répondre,Vous n’avez pas d’avis,"Oui, plutôt",Juillet 2021,Non,NaN,#NULL!,"Oui, mais vous ne l’utilisez que pour votre pa...",Facile,Difficile,Difficile,Difficile,Difficile,Facile,Facile,Difficile,Facile,Facile,Facile,Difficile,Facile,Difficile,Facile,Facile,Difficile,Difficile,Difficile,Difficile,Facile,Facile,Facile,Facile,"Les réseaux sociaux (Facebook, Twitter, etc.)",La radio,Les échanges avec vos proches,"Non, pas vraiment",Moins souvent,Vous votez rarement,Aucune formation politique,Aucune formation politique,Pas du tout confiance,Pas du tout confiance,Pas du tout confiance,Plutôt confiance,Plutôt confiance,Plutôt pas confiance,Plutôt pas confiance,Plutôt confiance,Plutôt confiance,Plutôt confiance,NaN,BTS DUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,0,Je ne sais pas,Je ne sais pas,Je ne sais pas,Je ne sais pas,Je ne sais pas,Je ne sais pas,Je ne sais pas,Je ne sais pas,1,Uncertain,Je ne sais pas,Favorable,Je ne sais pas,Je ne sais pas,Favorable,Défavorable à la vaccination en général,Première source,1.0,1,1.0,1.011978,-0.550123
3,1006,Complet,25416.857072,0.940896,Oui,1987,25-34,Une femme,Bourgogne-Franche-Comté,de 2 000 à moins de 20 000 habitants (exemple ...,Vous exercez une activit

In [177]:
# définir une fonction pour faire 5 groupes de Compétence politique
def competancepol_5groupes(x):
    if x < -0.782384 or x == -0.782384:
        return "Très élevé"
    if x > -0.782384 and x <= -0.372856:
        return "Elevé"
    if x > -0.372856 and x <= 0.065003:
        return "Moyen"
    if x > 0.065003 and x <= 0.848082 :
        return "Faible"
    if x > 0.848082:
        return "Très faible"

In [178]:
competpol_score['competpol_score']= competpol_score['Compoenent 0'].apply(lambda x : competancepol_5groupes(x))

In [179]:
competpol_score['competpol_score'].value_counts()

competpol_score
Elevé          534
Faible         438
Très faible    371
Moyen          370
Très élevé     309
Name: count, dtype: int64

In [180]:
#changer l'order des categories de la variable 'competpol_score'
competpol_score['competpol_score'] = pd.Categorical(competpol_score['competpol_score'], 
                            categories=['Très faible', 'Faible' ,'Moyen'
                                        ,'Elevé', 'Très élevé'], 
                            ordered=True)

In [181]:
# Variable 'Q42_A'
# Question: De laquelle des formations politiques suivantes vous sentez-vous le plus proche ou disons le moins éloigné ?
labels_data['Q42_A'].value_counts()

Q42_A
Aucune formation politique                                                 614
Le Rassemblement National                                                  250
Les Républicains                                                           214
La République en Marche                                                    208
Le Parti Socialiste                                                        155
Europe Ecologie Les Verts                                                  130
La France Insoumise de Jean-Luc Mélenchon                                  126
Une autre formation politique                                               61
Lutte Ouvrière                                                              54
Le MoDem                                                                    52
Debout La France de Nicolas Dupont-Aignan                                   32
Le Nouveau Parti Anticapitaliste d&apos;Olivier Besancenot                  29
Génération.s de Benoît Hamon                  

In [182]:
labels_data['Q42_A_cat'] = codes_data['Q42_A']

In [183]:
labels_data['Q42_A_cat'].value_counts()

Q42_A_cat
18    614
17    250
13    214
8     208
6     155
7     130
3     126
19     61
1      54
9      52
14     32
2      29
5      29
4      28
11     15
16      8
12      7
15      7
10      3
Name: count, dtype: int64

In [184]:
# définir une fonction pour recoder la proximité politique:
def proxpol(x):
    if x < 4 or x == 4:
        return "Far left"
    if x == 5 or x == 6 or x == 10:
        return "Left"
    if x == 7:
        return "Green party"
    if x == 8 or x == 9 or x == 11:
        return "Center"
    if x == 12 or x == 13:
        return "Right"
    if x == 14 or x == 15:
        return "Far right"
    if x == 16 or x == 17:
        return "Far right"
    if x == 18 or x == 19:
        return "No partisan proximity"
    if x == 99:
        return"Vous ne souhaitez pas répondre"

In [185]:
labels_data['Q42_A_cat']= labels_data['Q42_A_cat'].apply(lambda x : proxpol(x))

In [186]:
labels_data['Q42_A_cat'].value_counts()

Q42_A_cat
No partisan proximity    675
Far right                297
Center                   275
Far left                 237
Right                    221
Left                     187
Green party              130
Name: count, dtype: int64

In [187]:
labels_data['Q42_A_cat'] = labels_data['Q42_A_cat'].astype('category')

In [188]:
# Variable 'Q43_A'
# Question: Avez-vous confiance ou pas confiance dans la justice ?
labels_data['Q43_A'].value_counts()

Q43_A
Plutôt confiance                  868
Plutôt pas confiance              626
Pas du tout confiance             354
Tout à fait confiance             106
Vous n&apos;avez pas d’avis        54
Vous ne souhaitez pas répondre     14
Name: count, dtype: int64

In [189]:
# Variable 'Q43_B'
# Question: Avez-vous confiance ou pas confiance dans les médias ?
labels_data['Q43_B'].value_counts()

Q43_B
Plutôt pas confiance              818
Pas du tout confiance             642
Plutôt confiance                  438
Vous n&apos;avez pas d’avis        62
Tout à fait confiance              40
Vous ne souhaitez pas répondre     22
Name: count, dtype: int64

In [190]:
# Variable 'Q43_C'
# Question: Avez-vous confiance ou pas confiance dans la police ?
labels_data['Q43_C'].value_counts()

Q43_C
Plutôt confiance                  1162
Plutôt pas confiance               342
Tout à fait confiance              288
Pas du tout confiance              170
Vous n&apos;avez pas d’avis         45
Vous ne souhaitez pas répondre      15
Name: count, dtype: int64

In [191]:
# Variable 'Q43_D'
# Question: Avez-vous confiance ou pas confiance dans le Parlement ?
labels_data['Q43_D'].value_counts()

Q43_D
Plutôt pas confiance              697
Plutôt confiance                  651
Pas du tout confiance             453
Vous n&apos;avez pas d’avis       122
Tout à fait confiance              61
Vous ne souhaitez pas répondre     38
Name: count, dtype: int64

In [192]:
# Variable 'Q43_E'
# Question: Avez-vous confiance ou pas confiance dans la science ?
labels_data['Q43_E'].value_counts()

Q43_E
Plutôt confiance                  1301
Tout à fait confiance              336
Plutôt pas confiance               230
Vous n&apos;avez pas d’avis         72
Pas du tout confiance               58
Vous ne souhaitez pas répondre      25
Name: count, dtype: int64

In [193]:
# Variable 'Q43_F'
# Question: Avez-vous confiance ou pas confiance dans les grandes entreprises ?
labels_data['Q43_F'].value_counts()

Q43_F
Plutôt pas confiance              761
Plutôt confiance                  744
Pas du tout confiance             292
Vous n&apos;avez pas d’avis       141
Tout à fait confiance              63
Vous ne souhaitez pas répondre     21
Name: count, dtype: int64

In [194]:
# Variable 'Q43_G'
# Question: Avez-vous confiance ou pas confiance dans les agences gouvernementales qui contrôlent les risques liés à la santé et à l'environnement ?
labels_data['Q43_G'].value_counts()

Q43_G
Plutôt confiance                  814
Plutôt pas confiance              599
Pas du tout confiance             374
Tout à fait confiance             118
Vous n&apos;avez pas d’avis        89
Vous ne souhaitez pas répondre     28
Name: count, dtype: int64

In [195]:
# Variable 'Q43_H'
# Question: Avez-vous confiance ou pas confiance dans le gouvernement ?
labels_data['Q43_H'].value_counts()

Q43_H
Pas du tout confiance             622
Plutôt confiance                  620
Plutôt pas confiance              588
Tout à fait confiance              82
Vous n&apos;avez pas d’avis        77
Vous ne souhaitez pas répondre     33
Name: count, dtype: int64

In [196]:
# Variable 'Q43_I'
# Question: Avez-vous confiance ou pas confiance dans les médecins ?
labels_data['Q43_I'].value_counts()

Q43_I
Plutôt confiance                  1277
Tout à fait confiance              471
Plutôt pas confiance               154
Pas du tout confiance               53
Vous n&apos;avez pas d’avis         46
Vous ne souhaitez pas répondre      21
Name: count, dtype: int64

In [197]:
# Variable 'Q43_J'
# Question: Avez-vous confiance ou pas confiance dans les industriels du médicament ?
labels_data['Q43_J'].value_counts()

Q43_J
Plutôt confiance                  719
Plutôt pas confiance              679
Pas du tout confiance             396
Vous n&apos;avez pas d’avis       126
Tout à fait confiance              81
Vous ne souhaitez pas répondre     21
Name: count, dtype: int64

In [198]:
Items43 = ['Q43_A', 'Q43_B', 'Q43_C', 'Q43_D', 'Q43_E', 'Q43_F', 'Q43_G', 'Q43_H', 'Q43_I', 'Q43_J']

In [199]:
for i in labels_data[Items43].columns:
    labels_data[i] = pd.Categorical(labels_data[i], 
                            categories=['Pas du tout confiance', 'Plutôt pas confiance'
                                        , 'Plutôt confiance'
                                        , 'Tout à fait confiance'
                                        , 'Vous n&apos;avez pas d’avis'
                                       , 'Vous ne souhaitez pas répondre'] 
                                        , ordered=True)

In [200]:
for i in labels_data[Items43].columns:
    labels_data[i] =labels_data[i].cat.rename_categories({'Pas du tout confiance': "Pas du tout confiance"
                                                              , 'Plutôt pas confiance': "Plutôt pas confiance"
                                                              , 'Plutôt confiance' : "Plutôt confiance"
                                                              , 'Tout à fait confiance' : "Tout à fait confiance"
                                                              , 'Vous n&apos;avez pas d’avis' : "Vous n’avez pas d’avis"
                                                              , 'Vous ne souhaitez pas répondre' : "Vous ne souhaitez pas répondre"})

In [201]:
labels_data['Q14'].value_counts()

Q14
Vous connaissez un peu les essais cliniques                                    840
Vous connaissez seulement le terme « essai clinique »                          722
Vous connaissez bien les essais cliniques                                      244
Vous n&apos;avez jamais entendu parler d’essais cliniques avant aujourd’hui    107
Vous connaissez très bien les essais cliniques                                  87
Vous ne souhaitez pas répondre                                                  22
Name: count, dtype: int64

In [202]:
# Variable 'Q14'
labels_data[i] =labels_data[i].cat.rename_categories({'Vous n&apos;avez jamais entendu parler d’essais cliniques avant aujourd’hui': "Vous n’avez jamais entendu parler d’essais cliniques avant aujourd’hui"
                                                              , 'Vous connaissez seulement le terme « essai clinique »': "Vous connaissez seulement le terme « essai clinique »"
                                                              , 'Vous connaissez un peu les essais cliniques' : "Vous connaissez un peu les essais cliniques"
                                                              , 'Vous connaissez bien les essais cliniques' : "Vous connaissez bien les essais cliniques"
                                                              , 'Vous connaissez très bien les essais cliniques' : "Vous connaissez très bien les essais cliniques"
                                                              , 'Vous ne souhaitez pas répondre' : "Vous ne souhaitez pas répondre"})

In [203]:
data = pd.concat([labels_data, competpol_score['competpol_score']], axis = 1)
data

,CaseId,STATUT,poids_init,poids_redresses_stand,INT01,Q01,Q01_R0,Q02,Q03,Q04,Q05_A,Q05_B,Q05_R0,Q06,Q07,Q08,Q09,Q10_A,Q10_B,Q10_C,Q10_D,Q10_E,Q11,Q11_1,Q12,Q13,Q14,Q15,Q16_A,Q16_B,Q16_C,Q16_D,Q16_E,Q16_F,Q17_A,Q17_B,Q17_C,Q17_D,Q17_E,Q17_F,Q17_G,Q17_H,Q18,Q19,Q20,Q21,Q22,Q23_A,Q23_B,Q23_C,Q24_A,Q24_B,Q24_C,Q24_D,Q24_E,Q24_F,Q24_G,Q24_H,Q24_I,Q25,Q26,Q27,Q28_A,Q28_B,Q28_C,Q29,Q44,Q30,Q31,Q32,Q33,Q34,Q29_1,Q29_2,Q29_2_PRE,Q29_3,Q35,Q36_A,Q36_B,Q36_C,Q36_D,Q36_E,Q36_F,Q36_G,Q36_H,Q36_I,Q36_J,Q36_K,Q36_L,Q37_A,Q37_B,Q37_C,Q37_D,Q37_E,Q37_F,Q37_G,Q37_H,Q37_I,Q37_J,Q37_K,Q37_L,Q38M1,Q38M2,Q38M3,Q39,Q40,Q41,Q42_A,Q42_B,Q43_A,Q43_B,Q43_C,Q43_D,Q43_E,Q43_F,Q43_G,Q43_H,Q43_I,Q43_J,COMMENTAIRES,Diplome,Q17_A_cat,Q17_B_cat,Q17_C_cat,Q17_D_cat,Q17_E_cat,Q17_F_cat,Q17_G_cat,Q17_H_cat,Ind_CI,Q17_A_cat_No,Q17_B_cat_No,Q17_C_cat_No,Q17_D_cat_No,Q17_E_cat_No,Q17_F_cat_No,Q17_G_cat_No,Q17_H_cat_No,Ind_NCI,Q17_A_cat_NSP,Q17_B_cat_NSP,Q17_C_cat_NSP,Q17_D_cat_NSP,Q17_E_cat_NSP,Q17_F_cat_NSP,Q17_G_cat_NSP,Q17_H_cat_NSP,Ind_CI_NSP,Ind_CI_Cat,Ind_NCI_Cat,Ind_CI_NSP_Cat,Q17_A_cat_clustering,Q17_B_cat_clustering,Q17_C_cat_clustering,Q17_D_cat_clustering,Q17_E_cat_clustering,Q17_F_cat_clustering,Q17_G_cat_clustering,Q17_H_cat_clustering,cat_CI2,cluster_labels,Q30_cat,Q31_cat,Q32_cat,Q33_cat,Q34_cat,Ty_hes_vaccinale,Reseaux_sociaux_placement,Q39_cat,Q40_cat,Q41_cat,Q42_A_cat,competpol_score
0,10,Complet,25416.857072,1.015050,Oui,1946,65-74,Un homme,Provence-Alpes-Côte d&apos;Azur,de 2 000 à moins de 20 000 habitants (exemple ...,Vous êtes à la retraite ou en pré-retraite,Ingénieurs et cadres techniques d&apos;entreprise,Retraités,"Diplôme d’une grande école (ingénieur, commerc...",4000 euros et plus,2,Non,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,France,Oui,NaN,#NULL!,Vous connaissez un peu les essais cliniques,"Oui, plus ou moins",Tout à fait d’accord,Plutôt d’accord,Plutôt d’accord,Plutôt pas d’accord,Tout à fait d’accord,Plutôt pas d’accord,Plutôt,Plutôt,Tout à fait,Pas du tout,Plutôt,Pas du tout,Plutôt,Plutôt pas,Assez,A peu près autant de bien que de mal,"Oui, et vous pensez que la chloroquine et ses ...",Assez bon,"Oui, j’ai un problème de santé",Très facile,Facile,Facile,Tout à fait d’accord,"Ni d’accord, ni pas d’accord",Plutôt d’accord,Plutôt d’accord,Plutôt d’accord,Tout à fait d’accord,Plutôt d’accord,Plutôt d’accord,Plutôt d’accord,Vous connaissez seulement le terme “CBD”,Un peu,Jamais,Jamais,Tous les jours ou presque,Jamais,Vous êtes déjà vacciné (une dose ou plus),Plutôt pas favorable,Plutôt favorable,"Oui, tout à fait","Oui, plutôt","Oui, tout à fait","Oui, tout à fait",Mars 2021,Non,NaN,#NULL!,"Oui, mais vous l&apos;avez désinstallée",Difficile,Facile,Difficile,Très difficile,Difficile,Difficile,Difficile,Difficile,Facile,Difficile,Difficile,Difficile,Facile,Facile,Difficile,Facile,Difficile,Facile,Difficile,Difficile,Facile,Facile,Difficile,Difficile,Les échanges avec vos proches,La presse écrite imprimée,"Les réseaux sociaux (Facebook, Twitter, etc.)","Oui, beaucoup",Tous les jours ou presque,Vous votez à toutes les élections,Les Républicains,#NULL!,Plutôt pas confiance,Pas du tout confiance,Plutôt confiance,Plutôt pas confiance,Plutôt confiance,Plutôt confiance,Pas du tout confiance,Pas du tout confiance,Plutôt confiance,Plutôt confiance,NaN,Master ou plus,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,5.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3-4,1-2,0,D’accord,D’accord,D’accord,Pas d’accord,D’accord,Pas d’accord,D’accord,Pas d’accord,0,Money matters,Favorable,Favorable,Favorable,Favorable,Favorable,Non-hesitan,Troisième source,3.0,4,4.0,Right,Très élevé
1,1000,Complet,25416.857072,0.678083,Oui,2000,18-24,Une femme,Auvergne-Rhône-Alpes,de plus de 100 000 habitants (exemple : Grenob...,Vous exercez une activité professionnelle (act...,Professions intermédiaires administratives et ...,Professions Intermédiaires,"BTS, DUT, DEUST ou diplôme équivalent",1000 à 1500 euros,2,Non,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,France,Oui,NaN,#N

In [204]:
import pyshs

In [205]:
# fonction pour creer un tableau croisé multiple avec effectifs et pourcentages
def tableau_croise_final(
    data, var_dep, indeps):
    t_nombre = pyshs.tableau_croise_multiple(data,
                              var_dep,
                              indeps,
                              "poids_redresses_stand",               
                              contenu="absolu") 
    t_pourcentage = pyshs.tableau_croise_multiple(data,
                              var_dep,
                              indeps,
                              "poids_redresses_stand",                   
                              contenu="pourcentage" )
    t=pd.concat({n:pd.concat({"Effectif":t_nombre[n],"%":t_pourcentage[n]},axis=1)                     
        for n in t_nombre.columns if n != "Total" and n != "Distribution"},axis=1)
    
    for n in t_nombre.columns:
        if n == "Total":
            t["Total"] = t_nombre["Total"]

    t["Distribution"]= t_pourcentage["Distribution"]
    
    t = t.reset_index()
    
    t["Question"] = t["Variable"].apply(lambda x : x.split(' ')[0])
    
    t["Question"] = t["Question"].apply(lambda x : list({key: questions_variables[key] for key in [x]}.values())[0])
    
    t = t.set_index(['Question','Variable'])
    
    return t


   

In [206]:
data['Q14'].value_counts()

Q14
Vous connaissez un peu les essais cliniques                                    840
Vous connaissez seulement le terme « essai clinique »                          722
Vous connaissez bien les essais cliniques                                      244
Vous n&apos;avez jamais entendu parler d’essais cliniques avant aujourd’hui    107
Vous connaissez très bien les essais cliniques                                  87
Vous ne souhaitez pas répondre                                                  22
Name: count, dtype: int64

In [207]:
data['Q14'] =data['Q14'].cat.rename_categories({'Vous n&apos;avez jamais entendu parler d’essais cliniques avant aujourd’hui': "Vous n'avez jamais entendu parler d’essais cliniques avant aujourd’hui"
                                                              , 'Vous connaissez seulement le terme « essai clinique »': "Vous connaissez seulement le terme « essai clinique »"
                                                              , 'Vous connaissez un peu les essais cliniques' : "Vous connaissez un peu les essais cliniques"
                                                              , 'Vous connaissez bien les essais cliniques' : "Vous connaissez bien les essais cliniques"
                                                              , 'Vous connaissez très bien les essais cliniques' : "Vous connaissez très bien les essais cliniques"
                                                              , 'Vous ne souhaitez pas répondre' : "Vous ne souhaitez pas répondre"})

In [467]:
Tableau_variable_Q14 = tableau_croise_final(data, 'Q14', list_indeps)
Tableau_variable_Q14.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q14.xlsx")

<ipython-input-467-b1140547ae27>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q14.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q14.xlsx")


In [468]:
Tableau_variable_Q15 = tableau_croise_final(data, 'Q15', list_indeps)
Tableau_variable_Q15.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q15.xlsx")

<ipython-input-468-d41e00064a37>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q15.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q15.xlsx")


In [469]:
Tableau_variable_Q16_A = tableau_croise_final(data, 'Q16_A', list_indeps)
Tableau_variable_Q16_A.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q16_A.xlsx")

<ipython-input-469-472983f3f10f>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q16_A.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q16_A.xlsx")


In [470]:
Tableau_variable_Q16_B = tableau_croise_final(data, 'Q16_B', list_indeps)
Tableau_variable_Q16_B.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q16_B.xlsx")

<ipython-input-470-f08cd4b7dc85>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q16_B.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q16_B.xlsx")


In [471]:
Tableau_variable_Q16_C = tableau_croise_final(data, 'Q16_C', list_indeps)
Tableau_variable_Q16_C.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q16_C.xlsx")

<ipython-input-471-c85acca08077>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q16_C.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q16_C.xlsx")


In [472]:
Tableau_variable_Q16_D = tableau_croise_final(data, 'Q16_D', list_indeps)
Tableau_variable_Q16_D.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q16_D.xlsx")

<ipython-input-472-bc5981f5ae28>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q16_D.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q16_D.xlsx")


In [473]:
Tableau_variable_Q16_E = tableau_croise_final(data, 'Q16_E', list_indeps)
Tableau_variable_Q16_E.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q16_E.xlsx")

<ipython-input-473-0531fa2386f4>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q16_E.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q16_E.xlsx")


In [474]:
Tableau_variable_Q16_F = tableau_croise_final(data, 'Q16_F', list_indeps)
Tableau_variable_Q16_F.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q16_F.xlsx")

<ipython-input-474-1455692ea71e>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q16_F.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q16_F.xlsx")


In [475]:
Tableau_variable_Q17_A = tableau_croise_final(data, 'Q17_A', list_indeps)
Tableau_variable_Q17_A.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q17_A.xlsx")

<ipython-input-475-3d97d0c418fb>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q17_A.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q17_A.xlsx")


In [476]:
Tableau_variable_Q17_B = tableau_croise_final(data, 'Q17_B', list_indeps)
Tableau_variable_Q17_B.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q17_B.xlsx")

<ipython-input-476-e94454cd7992>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q17_B.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q17_B.xlsx")


In [477]:
Tableau_variable_Q17_C = tableau_croise_final(data, 'Q17_C', list_indeps)
Tableau_variable_Q17_C.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q17_C.xlsx")

<ipython-input-477-ef150c8fb9c8>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q17_C.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q17_C.xlsx")


In [478]:
Tableau_variable_Q17_D = tableau_croise_final(data, 'Q17_D', list_indeps)
Tableau_variable_Q17_D.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q17_D.xlsx")

<ipython-input-478-f7698c59995f>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q17_D.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q17_D.xlsx")


In [480]:
Tableau_variable_Q17_E = tableau_croise_final(data, 'Q17_E', list_indeps)
Tableau_variable_Q17_E.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q17_E.xlsx")

<ipython-input-480-9983e8bea892>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q17_E.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q17_E.xlsx")


In [481]:
Tableau_variable_Q17_F = tableau_croise_final(data, 'Q17_F', list_indeps)
Tableau_variable_Q17_F.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q17_F.xlsx")

<ipython-input-481-d02b20831fe9>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q17_F.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q17_F.xlsx")


In [482]:
Tableau_variable_Q17_G = tableau_croise_final(data, 'Q17_G', list_indeps)
Tableau_variable_Q17_G.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q17_G.xlsx")

<ipython-input-482-e7c2751a7470>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q17_G.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q17_G.xlsx")


In [483]:
Tableau_variable_Q17_H = tableau_croise_final(data, 'Q17_H', list_indeps)
Tableau_variable_Q17_H.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q17_H.xlsx")

<ipython-input-483-25efcc1ef712>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q17_H.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q17_H.xlsx")


In [484]:
Tableau_variable_Q20 = tableau_croise_final(data, 'Q20', list_indeps)
Tableau_variable_Q20.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q20.xlsx")

<ipython-input-484-e77fb209ea85>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q20.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q20.xlsx")


In [485]:
Tableau_variable_Q24_A = tableau_croise_final(data, 'Q24_A', list_indeps)
Tableau_variable_Q24_A.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q24_A.xlsx")

<ipython-input-485-53c4c141feb9>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q24_A.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q24_A.xlsx")


In [486]:
Tableau_variable_Q24_B = tableau_croise_final(data, 'Q24_B', list_indeps)
Tableau_variable_Q24_B.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q24_B.xlsx")

<ipython-input-486-3e0f2566bc7c>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q24_B.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q24_B.xlsx")


In [487]:
Tableau_variable_Q24_C = tableau_croise_final(data, 'Q24_C', list_indeps)
Tableau_variable_Q24_C.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q24_C.xlsx")

<ipython-input-487-f2defc44d9f5>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q24_C.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q24_C.xlsx")


In [488]:
Tableau_variable_Q24_D = tableau_croise_final(data, 'Q24_D', list_indeps)
Tableau_variable_Q24_D.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q24_D.xlsx")

<ipython-input-488-ef4919a0574a>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q24_D.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q24_D.xlsx")


In [489]:
Tableau_variable_Q24_E = tableau_croise_final(data, 'Q24_E', list_indeps)
Tableau_variable_Q24_E.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q24_E.xlsx")

<ipython-input-489-0a8d91c06b72>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q24_E.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q24_E.xlsx")


In [490]:
Tableau_variable_Q24_F = tableau_croise_final(data, 'Q24_F', list_indeps)
Tableau_variable_Q24_F.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q24_F.xlsx")

<ipython-input-490-dc402c30b9ae>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q24_F.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q24_F.xlsx")


In [491]:
Tableau_variable_Q24_G = tableau_croise_final(data, 'Q24_G', list_indeps)
Tableau_variable_Q24_G.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q24_G.xlsx")

<ipython-input-491-11142bfac73a>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q24_G.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q24_G.xlsx")


In [492]:
Tableau_variable_Q24_H = tableau_croise_final(data, 'Q24_H', list_indeps)
Tableau_variable_Q24_H.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q24_H.xlsx")

<ipython-input-492-5795047cbc59>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q24_H.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q24_H.xlsx")


In [493]:
Tableau_variable_Q24_I = tableau_croise_final(data, 'Q24_I', list_indeps)
Tableau_variable_Q24_I.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q24_I.xlsx")

<ipython-input-493-09bd20b4c6dd>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q24_I.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q24_I.xlsx")


In [494]:
Tableau_variable_Ind_CI_Cat = tableau_croise_final(data, 'Ind_CI_Cat', list_indeps)
Tableau_variable_Ind_CI_Cat.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Ind_CI_Cat.xlsx")

<ipython-input-494-bca2ef17f0bf>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Ind_CI_Cat.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Ind_CI_Cat.xlsx")


In [495]:
Tableau_variable_Ind_NCI_Cat = tableau_croise_final(data, 'Ind_NCI_Cat', list_indeps)
Tableau_variable_Ind_NCI_Cat.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Ind_NCI_Cat.xlsx")

<ipython-input-495-3c9a599db978>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Ind_NCI_Cat.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Ind_NCI_Cat.xlsx")


In [496]:
Tableau_variable_Ind_CI_NSP_Cat = tableau_croise_final(data, 'Ind_CI_NSP_Cat', list_indeps)
Tableau_variable_Ind_CI_NSP_Cat.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Ind_CI_NSP_Cat.xlsx")

<ipython-input-496-9d97acd7b0b6>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Ind_CI_NSP_Cat.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Ind_CI_NSP_Cat.xlsx")


In [497]:
Tableau_variable_cluster_labels = tableau_croise_final(data, 'cluster_labels', list_indeps)
Tableau_variable_cluster_labels.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_cluster_labels.xlsx")

<ipython-input-497-8980decc1f1d>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_cluster_labels.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_cluster_labels.xlsx")


In [498]:
Tableau_variable_Q29 = tableau_croise_final(data, 'Q29', list_indeps)
Tableau_variable_Q29.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q29.xlsx")

<ipython-input-498-8b75fe4fb8ff>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q29.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q29.xlsx")


In [499]:
Tableau_variable_Q30 = tableau_croise_final(data, 'Q30', list_indeps)
Tableau_variable_Q30.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q30.xlsx")

<ipython-input-499-3e582b55a63e>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q30.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q30.xlsx")


In [208]:
Tableau_variable_Q31 = tableau_croise_final(data, 'Q31', list_indeps)
Tableau_variable_Q31.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q31.xlsx")

<ipython-input-208-d4f1dfa73fc0>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q31.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q31.xlsx")


In [209]:
Tableau_variable_Q32 = tableau_croise_final(data, 'Q32', list_indeps)
Tableau_variable_Q32.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q32.xlsx")

<ipython-input-209-00dd93fe496e>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q32.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q32.xlsx")


In [210]:
Tableau_variable_Q33 = tableau_croise_final(data, 'Q33', list_indeps)
Tableau_variable_Q33.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q33.xlsx")

<ipython-input-210-e78b3a220243>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q33.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q33.xlsx")


In [211]:
Tableau_variable_Q34 = tableau_croise_final(data, 'Q34', list_indeps)
Tableau_variable_Q34.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q34.xlsx")

<ipython-input-211-f254164c689b>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q34.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q34.xlsx")


In [212]:
Tableau_variable_Ty_hes_vaccinale = tableau_croise_final(data, 'Ty_hes_vaccinale', list_indeps)
Tableau_variable_Ty_hes_vaccinale.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Ty_hes_vaccinale.xlsx")

<ipython-input-212-e76cba9e184a>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Ty_hes_vaccinale.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Ty_hes_vaccinale.xlsx")


In [213]:
Tableau_variable_Q43_G = tableau_croise_final(data, 'Q43_G', list_indeps)
Tableau_variable_Q43_G.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q43_G.xlsx")

<ipython-input-213-196d5ecbf8bc>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q43_G.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q43_G.xlsx")


In [214]:
Tableau_variable_Q43_I = tableau_croise_final(data, 'Q43_I', list_indeps)
Tableau_variable_Q43_I.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q43_I.xlsx")

<ipython-input-214-98ddadfce994>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q43_I.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q43_I.xlsx")


In [215]:
Tableau_variable_Q43_J = tableau_croise_final(data, 'Q43_J', list_indeps)
Tableau_variable_Q43_J.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q43_J.xlsx")

<ipython-input-215-3619f2e97959>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  Tableau_variable_Q43_J.to_excel(r"C:\Users\rana_\OneDrive\Bureau\inserm\travaux\Slavaco3\Tableaux_croisés\Tableau_variable_Q43_J.xlsx")
